In [1]:
# Imports
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False #disable ugly white lines which are present in google colab for matplotlib
import numpy as np
import datetime
today = datetime.datetime.today() #To work with datetime values. Only relative time matters in this project, so selecting a random date is not a problem. 
from sklearn.metrics import classification_report,auc,r2_score,matthews_corrcoef
import shap
from catboost import CatBoostClassifier,CatBoostRegressor,Pool
from catboost.utils import get_roc_curve
import xgboost as xgb
import pandas as pd
import re
from tqdm import tqdm
from tabulate import tabulate
from scipy.stats import linregress,ttest_ind,ranksums
pd.options.display.float_format = '{:20,.2f}'.format
np.set_printoptions(suppress=True)
from numpy.random import RandomState
data_amsterdam_get_path = "../../IC_DC_AF/Data/"
data_amsterdam_save_path = "../data/amsterdam/"

## Load the AmsterdamUMCdb dictionary. 
Use the **Filter** button to quickly browse through the dictionary.

In [2]:
#get the amsterdamumcdb package from PiPy repository for use in Colab
#!pip install amsterdamumcdb
import amsterdamumcdb as adb

dictionary = adb.get_dictionary()
dictionary

,itemid,item,item_en,abbreviation,categoryid,category,category_en,ordercategoryid,ordercategory,islabresult,valueid,value,unitid,unit,low_normal_value,high_normal_value,table,count,count_validated
0,7321,Soluvit solution,NaN,SoluvitOpl,276,Additieven,additives,67.00,Injecties Hormonen/Vitaminen/Mineralen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drugitems,25,NaN
1,12096,ADL Hulp,NaN,Hulp,582,Anamnese-ADL,NaN,NaN,NaN,0.00,1.00,Burenhulp,NaN,NaN,NaN,NaN,listitems,12,12.00
2,12096,ADL Hulp,NaN,Hulp,582,Anamnese-ADL,NaN,NaN,NaN,0.00,2.00,Thuiszorg,NaN,NaN,NaN,NaN,listitems,155,155.00
3,12096,ADL Hulp,NaN,Hulp,582,Anamnese-ADL,NaN,NaN,NaN,0.00,3.00,Kraamhulp,NaN,NaN,NaN,NaN,listitems,1,1.00
4,12099,Allergie 1 Bron,NaN,Bron (1),473,Anamnese-Allergieen&Risicofactoren,NaN,NaN,NaN,0.00,1.00,Anamnestisch,NaN,NaN,NaN,NaN,listitems,1153,"1,153.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9026,8883,Airtrap,NaN,Airtrap,331,ZZ-OUD-Resp-PatientGeg.,NaN,NaN,NaN,0.00,NaN,NaN,6.00,ml,NaN,NaN,numericitems,1508,"1,476.00"
9027,8884,End Tidal CO2 %,end tidal carbon dioxide partial pressure,End Tidal CO2 %,331,ZZ-OUD-Resp-PatientGeg.,NaN,NaN,NaN,0.00,NaN,NaN,0.00,None,NaN,NaN,numericitems,64,58.00
9028,8885,End Tidal CO2 mmHG,end tidal carbon dioxide partial pressure,End Tidal CO2,331,ZZ-OUD-Resp-PatientGeg.,NaN,NaN,NaN,0.00,NaN,NaN,173.00,mmHg,NaN,NaN,numericitems,409,402.00
9029,8886,Beademings Temperatuur,NaN,Beadm. Temp,331,ZZ-OUD-Resp-PatientGeg.,NaN,NaN,NaN,0.00,NaN,NaN,59.00,°C,NaN,NaN,numericitems,67,67.00


In [3]:
dictionary[dictionary.item.str.contains("vanco.*",flags=re.IGNORECASE, regex=True)]#.sort_values("count",ascending=False)


,itemid,item,item_en,abbreviation,categoryid,category,category_en,ordercategoryid,ordercategory,islabresult,valueid,value,unitid,unit,low_normal_value,high_normal_value,table,count,count_validated
2859,7199,Vancomycine dal spiegel,NaN,Vanco_dalS,277,LAB TOXICOLOGIE,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,5,10,numericitems,10,10.00
2866,8347,Vancomycine top spiegel,NaN,Vanco_topS,277,LAB TOXICOLOGIE,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,20,30,numericitems,1,1.00
3889,9835,Vancomycine (bloed),NaN,Vancomycine (bloed),482,LAB-Tox-bloed,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,NaN,NaN,numericitems,3023,"3,023.00"
3890,9837,VANCOMYdal (bloed),NaN,VANCOMYdal (bloed),482,LAB-Tox-bloed,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,3,8,numericitems,29,29.00
3891,9839,VANCOMYtop (bloed),NaN,VANCOMYtop (bloed),482,LAB-Tox-bloed,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,20,30,numericitems,1,1.00
3957,9836,VANCOMYonb (overig),NaN,VANCOMYonb (overig),483,LAB-Tox-diversen,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,NaN,NaN,numericitems,24,24.00
3960,18568,Vancomycine (liquor),NaN,Vancomycine (liquor),504,LAB-Tox-liquor,NaN,NaN,NaN,1.00,NaN,NaN,63.00,mg/l,NaN,NaN,numericitems,1,1.00
4766,7064,Vancomycine,NaN,Vancomycine,268,Medicatie,NaN,65.00,2. Spuitpompen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drugitems,22,NaN
4767,7064,Vancomycine,NaN,Vancomycine,268,Medicatie,NaN,21.00,Niet iv Antimicrobiele middelen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drugitems,1886,NaN
4768,7064,Vancomycine,NaN,Vancomycine,268,Medicatie,NaN,15.00,Injecties Antimicrobiele middelen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drugitems,23868,NaN


# Extraction code

In [ ]:
# Feature_list_prediction_model = pd.read_csv(r"Data/Feature_list_prediction_model.csv",sep=',')
Feature_list_prediction_model = pd.read_csv(r"Data/Extracted/AmsterdamUMCdb_Feature_list_prediction_model_final.csv",sep=';')
# Feature_list_prediction_model = Feature_list_prediction_model[Feature_list_prediction_model["To Be Evaluated"]=="Yes"]
feats_pd = Feature_list_prediction_model[Feature_list_prediction_model["count"]<250].drop_duplicates("itemid")

In [ ]:
#This code takes around 30 seconds
afib_pd = None

for chunk in tqdm.tqdm(pd.read_csv(r"Data/listitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):
        if afib_pd is None:
            afib_pd = chunk[(chunk.itemid==6671)&(chunk.valueid==13)]
        else:
            afib_pd = pd.concat([afib_pd,chunk[(chunk.itemid==6671)&(chunk.valueid==13)]])

afib_pd.to_csv("Data/Extracted/afib.csv")


In [ ]:
listfeatures_pd = None

list_features = [10469, 10697, 10702, 12128, 18588, 16997,
       19732]

for chunk in tqdm.tqdm(pd.read_csv(r"Data/listitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):
        if listfeatures_pd is None:
            listfeatures_pd = chunk[chunk.itemid.isin(list_features)]
        else:
            listfeatures_pd = pd.concat([listfeatures_pd,chunk[chunk.itemid.isin(list_features)]])

listfeatures_pd.to_csv("Data/Extracted/listfeatures_new.csv",index=False)

In [ ]:
#This code takes 20 minutes, 490 iterations
#around 490*2.000.000 rows
fluidbalance_numeric_pd = None

for file in tqdm.tqdm(os.listdir(r"Data/Extracted/numericitems/")):
    chunk =  pd.read_csv("Data/Extracted/numericitems/"+file,sep=',',encoding='latin-1',low_memory=False)
    if fluidbalance_numeric_pd is None:
        fluidbalance_numeric_pd = chunk[(~chunk.fluidout.isna()) & (chunk.fluidout!=0) & (chunk.measuredat>=0)][["admissionid","fluidout","measuredat"]]
    else:
        fluidbalance_numeric_pd = pd.concat([fluidbalance_numeric_pd,chunk[(~chunk.fluidout.isna()) & (chunk.fluidout!=0) & (chunk.measuredat>=0)][["admissionid","fluidout","measuredat"]]])

    if it == 13:

        if os.path.exists("Data/Extracted/fluidbalance_numeric.csv"):
            fluidbalance_numeric_pd.to_csv("Data/Extracted/fluidbalance_numeric.csv",mode='a',index=False, header=False)
        else:
            fluidbalance_numeric_pd.to_csv("Data/Extracted/fluidbalance_numeric.csv",index=False)

        heart_rate_pd = None
        fluidbalance_numeric_pd = None

        it = 0
    else:
        it = it + 1

In [ ]:
numeric_items_af_features = None

NOR_Causal_features = [#9476,#Is in procedureorderitems
                    15808,#is in list items
                    #7480, #Propofol is in drugitems
                    #7214, #Etomidaat is in drugitems
                    7666,
                    6642,
                    6640,
                    8843,
                    9580,
                    10053,
                    14058]
                    #12465#is in processitems]


for itemid in tqdm.tqdm(NOR_Causal_features,ascii=True):
    if os.path.exists("Data/Extracted/numericitems/"+str(itemid)+".csv"):
        if numeric_items_af_features is None:
            numeric_items_af_features = pd.read_csv("Data/Extracted/numericitems/"+str(itemid)+".csv")[["admissionid", "value","measuredat", "itemid"]]
        else:
            numeric_items_af_features = pd.concat([numeric_items_af_features,pd.read_csv("Data/Extracted/numericitems/"+str(itemid)+".csv")[["admissionid", "value","measuredat", "itemid"]]])
print("SAVING")
numeric_items_af_features.to_csv("Data/Extracted/NOR_Causal_numeric_items.csv",index=False)
  
    
numeric_items_af_features = None

PEEP_Causal_features = [13076,                  
                    8859,
                    #7480, #Propofol is in drugitems
                    #7214, #Etomidaat is in drugitems
                    8860,
                    6848,
                    8903,
                    9990,
                    9996,
                    10282,                 
                    12310,
                    12311,
                    21213,
                    21214,
                    6699,
                    8845,
                    12279,
                    15142,#is in list items
                    8874,
                    12266]
                       

for itemid in tqdm.tqdm(PEEP_Causal_features,ascii=True):
    if os.path.exists("Data/Extracted/numericitems/"+str(itemid)+".csv"):
        if numeric_items_af_features is None:
            numeric_items_af_features = pd.read_csv("Data/Extracted/numericitems/"+str(itemid)+".csv")[["admissionid", "value","measuredat", "itemid"]]
        else:
            numeric_items_af_features = pd.concat([numeric_items_af_features,pd.read_csv("Data/Extracted/numericitems/"+str(itemid)+".csv")[["admissionid", "value","measuredat", "itemid"]]])
print("SAVING")
numeric_items_af_features.to_csv("Data/Extracted/PEEP_Causal_numeric_items.csv",index=False)

In [ ]:
#The code loop will take around 7 minutes, saving another 5 minutes
numeric_items_af_features = None

numeric_items_itemids = [6640,#Hartfrequentie (bloed) / Heartrate
    8662,  #Temperatuur Perifeer 1 never used in features
    8659, #(perifeer 2) never used in features
    12106, #(risico roken sig/dag) never used in features
    6642, #(ABP Gemiddeld)
    6641, #(ABP Systolic)
    8794, #(UrineCAD)
    8796, #(UrineSupraPubis) never used in features
    8798, #(Urine Spontaan) never used in features
    8803, #(UrineUP) never used in features
    8800, #(urine incontinentie) never used in features
    10743, #(Nefrodrain li Uit) never used in features
    10745, #(Nefrodrain re Uit) never used in features
    10187, #(Vrij-T4) never used in features
    6839, #(Magnesium) never used in features
    14249, #(NT-proBNP (bloed)) never used in features
    9927, #(Kalium (bloed)) never used in features
    9556, #(Kalium Astrup) never used in features
    6835, #(Kalium) never used in features
    9924, #(Natrium (bloed)) never used in features
    9555, #(Natrium Astrup) never used in features
    6840, #(Natrium) never used in features
    9580, #Laktaat Astrup never used in features
    6837, #Laktaat never used in features
    10053, #Lactaat (bloed) never used in features
    9941, #(Kreatinine (bloed))
    6836, #(Kreatinine)
    10286, #(Hb(v.Bgs) (bloed))
    9960, #(Hb (bloed) )
    6778,  #(Hemoglobine)
    9553, #(CtHB Astrup)
    12310, #(ph (bloed))
    #8658, # Temp Bloed  never used in features
    #13063, # Temp Huid use only one of temp, to save RAM, if temp bloed is relevant, we can add temp huid
    6848, #(PH)
    6846, #(PCO2)
    9990, #(pCo2 (bloed))
    21213, #(PCO2 (bloed) -kPa))
    7433, #(PO2)
    10079, # CRP (bloed)
    12284, # PEEP (set)
    9996, #(po2 (bloed))
    21214, #(PO2 (bloed) -kPa))
    11978, #(ALAT (bloed))
    6800, #(ALAT)
    11978, #(ALAT (bloed))
    6806, #(ASAT)
    11990, #(ASAT (bloed))
    6833, #(Glucose Bloed))
    9947, #(Glucose (bloed))
    9557, #(Glucose Astrup)
    9558, #(Chloor Astrup)  never used in features
    6819, #(Chloor) never used in features
    9930, #(Chloor (bloed)) never used in features
    14413, #(Cl (onv.ISE) (bloed)) 
    9954, # Cholesterol (bloed) never used in features
    10057, # LDL (Ber. bloed) never used in features
    10056, # Hdl-Chol (bloed) never used in features
    9956, # Triglyceriden (bloed)
    12282, # O2 concentratie (Set)
    8845, # O2 l/min
    11998, # CK Blood
    6655,#From here on contains the new features
    6656,
    6779,
    6789,
    6797,
    6803,
    6807,
    6810,
    6812,
    6813,
    6817,
    6822,
    6824,6825,
    6828,
    6831,
    6838,
    6850,
    8115,
    8862,
    8879,
    9559,
    9560,
    9561,
    9933,
    9935,
    9937,
    9943,
    9945,
    9952,
    9962,
    9964,
    9965,
    9989,
    9992,
    9994,
    10051,
    10175,
    10195,
    10201,
    10238,
    10267,
    10282,
    10393,
    10407,
    10736,
    11812,
    11893,
    11894,
    11902,
    11925,
    11944,
    11984,
    12043,
    12071,
    12079,
    12087,
    12107,
    12311,
    12444,
    12460,
    13151,
    13952,
    14047,
    14055,
    14058,
    14849,
    17982,
    18361,
    19227,
    19500,
    19750,
    20078,
    20079]

for itemid in tqdm.tqdm(numeric_items_itemids,ascii=True):
    if os.path.exists("Data/Extracted/numericitems/"+str(itemid)+".csv"):
        if numeric_items_af_features is None:
            numeric_items_af_features = pd.read_csv("Data/Extracted/numericitems/"+str(itemid)+".csv")[["admissionid", "value","measuredat", "itemid"]]
        else:
            numeric_items_af_features = pd.concat([numeric_items_af_features,pd.read_csv("Data/Extracted/numericitems/"+str(itemid)+".csv")[["admissionid", "value","measuredat", "itemid"]]])
print("SAVING")
numeric_items_af_features.to_csv("Data/Extracted/numeric_items_af_features_new.csv",index=False)

In [ ]:
#There are 98 iterations with this code. This will take 2 hours
for chunk in tqdm.tqdm(pd.read_csv(r"Data/numericitems.csv",sep=',',
                             chunksize=10000000, 
                             iterator=True, encoding='latin-1',low_memory=False),ascii=True):
        
        for itemid in chunk.itemid.unique():
            path = "Data/Extracted/numericitems/"+str(itemid)+".csv"
            
            if os.path.exists(path):
                chunk[chunk.itemid==itemid].to_csv(path,mode='a',index=False, header=False)
            else:
                chunk[chunk.itemid==itemid].to_csv(path,index=False)
            
        chunk = None


In [ ]:
featrs = np.array([10469, 10697, 10702, 12128, 18588, 16997,6640,#Hartfrequentie (bloed) / Heartrate
    8662,  #Temperatuur Perifeer 1 never used in features
    8659, #(perifeer 2) never used in features
    12106, #(risico roken sig/dag) never used in features
    6642, #(ABP Gemiddeld)
    6641, #(ABP Systolic)
    8794, #(UrineCAD)
    8796, #(UrineSupraPubis) never used in features
    8798, #(Urine Spontaan) never used in features
    8803, #(UrineUP) never used in features
    8800, #(urine incontinentie) never used in features
    10743, #(Nefrodrain li Uit) never used in features
    10745, #(Nefrodrain re Uit) never used in features
    10187, #(Vrij-T4) never used in features
    6839, #(Magnesium) never used in features
    14249, #(NT-proBNP (bloed)) never used in features
    9927, #(Kalium (bloed)) never used in features
    9556, #(Kalium Astrup) never used in features
    6835, #(Kalium) never used in features
    9924, #(Natrium (bloed)) never used in features
    9555, #(Natrium Astrup) never used in features
    6840, #(Natrium) never used in features
    9580, #Laktaat Astrup never used in features
    6837, #Laktaat never used in features
    10053, #Lactaat (bloed) never used in features
    9941, #(Kreatinine (bloed))
    6836, #(Kreatinine)
    10286, #(Hb(v.Bgs) (bloed))
    9960, #(Hb (bloed) )
    6778,  #(Hemoglobine)
    9553, #(CtHB Astrup)
    12310, #(ph (bloed))
    8658, # Temp Bloed  never used in features
    13063, # Temp Huid use only one of temp, to save RAM, if temp bloed is relevant, we can add temp huid
    6848, #(PH)
    6846, #(PCO2)
    9990, #(pCo2 (bloed))
    21213, #(PCO2 (bloed) -kPa))
    7433, #(PO2)
    10079, # CRP (bloed)
    12284, # PEEP (set)
    9996, #(po2 (bloed))
    21214, #(PO2 (bloed) -kPa))
    11978, #(ALAT (bloed))
    6800, #(ALAT)
    11978, #(ALAT (bloed))
    6825,
    6806, #(ASAT)
    11990, #(ASAT (bloed))
    6833, #(Glucose Bloed))
    9947, #(Glucose (bloed))
    9557, #(Glucose Astrup)
    9558, #(Chloor Astrup)  never used in features
    6819, #(Chloor) never used in features
    9930, #(Chloor (bloed)) never used in features
    14413, #(Cl (onv.ISE) (bloed)) 
    9954, # Cholesterol (bloed) never used in features
    10057, # LDL (Ber. bloed) never used in features
    10056, # Hdl-Chol (bloed) never used in features
    9956, # Triglyceriden (bloed)
    12282, # O2 concentratie (Set)
    8845, # O2 l/min
    11998, # CK Blood
    6655,#From here on contains the new features
    6656,
    6779,
    6789,
    6797,
    6803,
    6807,
    6810,
    6812,
    6813,
    6817,
    6822,
    6824,
    6828,
    6831,
    6838,
    6850,
    8115,
    8862,
    8879,
    9559,
    9560,
    9561,
    9933,
    9935,
    9937,
    9943,
    9945,
    9952,
    9962,
    9964,
    9965,
    9989,
    9992,
    9994,
    10051,
    10175,
    10195,
    10201,
    10238,
    10267,
    10282,
    10393,
    10407,
    10736,
    11812,
    11893,
    11894,
    11902,
    11925,
    11944,
    11984,
    12043,
    12071,
    12079,
    12087,
    12107,
    12311,
    12444,
    12460,
    13151,9466,6825,
    13952,
    14047,
    14055,
    14058,
    14849,
    17982,
    18361,
    19227,
    19500,
    19750,7229, #-- Noradrenaline
      7244, #-- Furosemide
      7219, #--Fentanyl
      9543, #-- Magnesium Oxide
      7027, #-- Thiamine (Vitamine B1)
                          #new feature list
        7295,  8940, 10739, 19932,  6818,  6844,  6862,  6864,  
        6933,  7006,  7106,  7135,    7148,  7151,  7157,
         7174,  7178,  7179,   7194,  7196,  7214,  
        7225,  7412,  7480,  7624,  8020,  8470,  8998,  8999,  9001,
        9002,  9014,  9015,  9087,  9139, 12938, 16113, 18783, 18807,
       19129,
    20078,
    20079,19732])

len(np.sort(featrs))

In [ ]:
temp = Feature_list_prediction_model[Feature_list_prediction_model["count"]>250].drop_duplicates("itemid")

not_in_itemids = []
for itemids in featrs:
    if not int(itemids) in temp.itemid.values:
        not_in_itemids.append(itemids)

not_in_itemids

In [ ]:
temp = Feature_list_prediction_model[Feature_list_prediction_model["count"]>1].drop_duplicates("itemid")

In [ ]:
#This code takes around 20 seconds
drug_items_pd = None
drugitems_fluidbalance_pd = None

drug_items_feature_ist = [7229, #-- Noradrenaline
      7244, #-- Furosemide
      7219, #--Fentanyl
      9543, #-- Magnesium Oxide
      7027, #-- Thiamine (Vitamine B1)
                          #new feature list
        7295,  8940, 10739, 19932,  6818,  6844,  6862,  6864,  
                6933,  7006,  7106,  7135,    7148,  7151,  7157,
         7174,  7178,  7179,    7194,  7196,  7214,  
        7225,  7412,  7480,  7624,  8020,  8470,  8998,  8999,  9001,
        9002,  9014,  9015,  9087,  9139, 12938, 16113, 18783, 18807,
       19129                  ]

for chunk in tqdm.tqdm(pd.read_csv(r"Data/drugitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):
        if drug_items_pd is None:
            drug_items_pd = chunk[chunk.itemid.isin(drug_items_feature_ist)]
        else:
            drug_items_pd = pd.concat([drug_items_pd,chunk[chunk.itemid.isin(drug_items_feature_ist)]])
            
        if drugitems_fluidbalance_pd is None:
            drugitems_fluidbalance_pd = chunk[(chunk.fluidin!=0)&(chunk.duration>=1)][["admissionid","start","stop","fluidin","dose","solutionadministered","duration","rate"]]
        else:
            drugitems_fluidbalance_pd = pd.concat([drugitems_fluidbalance_pd,chunk[(chunk.fluidin!=0)&(chunk.duration>=1)][["admissionid","start","stop","fluidin","dose","solutionadministered","duration","rate"]]])

drug_items_pd.to_csv("Data/Extracted/drug_items_features_new.csv",index=False)
drugitems_fluidbalance_pd.to_csv("Data/Extracted/drugitems_fluidbalance.csv",index=False)

In [ ]:
#This code takes around 20 seconds
procedureorderitems_pd = None

procedureorderitems_list = [9476]#9466 = start van CVVH, is zoals 9476 maar kleiner

for chunk in tqdm.tqdm(pd.read_csv(r"Data/procedureorderitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):
        if procedureorderitems_pd is None:
            procedureorderitems_pd = chunk[chunk.itemid.isin(procedureorderitems_list)]
        else:
            procedureorderitems_pd = pd.concat([procedureorderitems_pd,chunk[chunk.itemid.isin(procedureorderitems_list)]])

procedureorderitems_pd.to_csv("Data/Extracted/procedureorderitems_features.csv",index=False)


In [ ]:
#Takes around 5 minutes
diagnosis_groups = None

for chunk in tqdm.tqdm(pd.read_csv(r"Data/listitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):


    diagnosis_groups_query = """
    SELECT admissionid,
            item, 
            CASE
                WHEN itemid IN (
                    18669, --NICE APACHEII diagnosen
                    18671 --NICE APACHEIV diagnosen
                )
                THEN substr(value, 0, instr(value, ' - '))
                -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Non-operative cardiovascular
                ELSE value
            END as diagnosis_group,
            valueid as diagnosis_group_id,
            ROW_NUMBER() OVER(PARTITION BY admissionid
            ORDER BY 
                CASE --prefer NICE > APACHE IV > II > D
                    WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                    WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                    WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                    WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                    WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                    WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
                END DESC,
            measuredat DESC) AS rownum
        FROM chunk
        WHERE itemid IN (
            --MAIN GROUP - LEVEL 0
            13110, --D_Hoofdgroep
            16651, --DMC_Hoofdgroep, Medium Care

            18588, --Apache II Hoofdgroep
            16997, --APACHE IV Groepen

            18669, --NICE APACHEII diagnosen
            18671 --NICE APACHEIV diagnosen
        )
    """
    diagnosis_groups_chunk = ps.sqldf(diagnosis_groups_query)
    
    if diagnosis_groups is None:
        diagnosis_groups = diagnosis_groups_chunk
    else: 
        diagnosis_groups = pd.concat([diagnosis_groups,diagnosis_groups_chunk])
        
diagnosis_groups.to_csv("Data/Extracted/combined_diagnosis/diagnosis_groups.csv",index=False)

In [ ]:
#Takes around 6 minutes
diagnosis_subgroups = None

for chunk in tqdm.tqdm(pd.read_csv(r"Data/listitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):

    diagnosis_subgroups_query = """
    SELECT admissionid,
            item, 
            value as diagnosis_subgroup,
            valueid as diagnosis_subgroup_id,
            ROW_NUMBER() OVER(PARTITION BY admissionid
            ORDER BY measuredat DESC) AS rownum
        FROM chunk
        WHERE itemid IN (
            --SUB GROUP - LEVEL 1
            13111, --D_Subgroep_Thoraxchirurgie
            16669, --DMC_Subgroep_Thoraxchirurgie
            13112, --D_Subgroep_Algemene chirurgie
            16665, --DMC_Subgroep_Algemene chirurgie
            13113, --D_Subgroep_Neurochirurgie
            16667, --DMC_Subgroep_Neurochirurgie
            13114, --D_Subgroep_Neurologie
            16668, --DMC_Subgroep_Neurologie
            13115, --D_Subgroep_Interne geneeskunde
            16666 --DMC_Subgroep_Interne geneeskunde
        )
    """

    diagnosis_subgroups_chunk = ps.sqldf(diagnosis_subgroups_query)
    
    if diagnosis_subgroups is None:
        diagnosis_subgroups = diagnosis_subgroups_chunk
    else: 
        diagnosis_subgroups = pd.concat([diagnosis_subgroups,diagnosis_subgroups_chunk])
        
diagnosis_subgroups.to_csv("Data/Extracted/combined_diagnosis/diagnosis_subgroups.csv",index=False)


In [ ]:
#Takes around 6 minutes
diagnosis = None

for chunk in tqdm.tqdm(pd.read_csv(r"Data/listitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):

    diagnosis_query = """
    SELECT admissionid,
            item, 
            CASE
                WHEN itemid IN (
                    18669, --NICE APACHEII diagnosen
                    18671 --NICE APACHEIV diagnosen
                )
                THEN substr(value, instr(value, ' - ')+1) --split_part(value, ' - ', 2) 
                -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Anaphylaxis
                ELSE value
            END as diagnosis,
            CASE
                WHEN itemid IN (
                    --SURGICAL
                    13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
                    16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
                    13117, --D_Thoraxchirurgie_Cardio anders
                    16672, --DMC_Thoraxchirurgie_Cardio anders
                    13118, --D_Thoraxchirurgie_Aorta chirurgie
                    16670, --DMC_Thoraxchirurgie_Aorta chirurgie
                    13119, --D_Thoraxchirurgie_Pulmonale chirurgie
                    16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

                    --Not surgical: 13141, --D_Algemene chirurgie_Algemeen   
                    --Not surgical: 16642, --DMC_Algemene chirurgie_Algemeen
                    13121, --D_Algemene chirurgie_Buikchirurgie
                    16643, --DMC_Algemene chirurgie_Buikchirurgie
                    13123, --D_Algemene chirurgie_Endocrinologische chirurgie
                    16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
                    13145, --D_Algemene chirurgie_KNO/Overige
                    16645, --DMC_Algemene chirurgie_KNO/Overige
                    13125, --D_Algemene chirurgie_Orthopedische chirurgie
                    16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
                    13122, --D_Algemene chirurgie_Transplantatie chirurgie
                    16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
                    13124, --D_Algemene chirurgie_Trauma
                    16648, --DMC_Algemene chirurgie_Trauma
                    13126, --D_Algemene chirurgie_Urogenitaal
                    16649, --DMC_Algemene chirurgie_Urogenitaal
                    13120, --D_Algemene chirurgie_Vaatchirurgie
                    16650, --DMC_Algemene chirurgie_Vaatchirurgie

                    13128, --D_Neurochirurgie _Vasculair chirurgisch
                    16661, --DMC_Neurochirurgie _Vasculair chirurgisch
                    13129, --D_Neurochirurgie _Tumor chirurgie
                    16660, --DMC_Neurochirurgie _Tumor chirurgie
                    13130, --D_Neurochirurgie_Overige
                    16662, --DMC_Neurochirurgie_Overige

                    18596, --Apache II Operatief  Gastr-intenstinaal
                    18597, --Apache II Operatief Cardiovasculair
                    18598, --Apache II Operatief Hematologisch
                    18599, --Apache II Operatief Metabolisme
                    18600, --Apache II Operatief Neurologisch
                    18601, --Apache II Operatief Renaal
                    18602, --Apache II Operatief Respiratoir

                    17008, --APACHEIV Post-operative cardiovascular
                    17009, --APACHEIV Post-operative gastro-intestinal
                    17010, --APACHEIV Post-operative genitourinary
                    17011, --APACHEIV Post-operative hematology
                    17012, --APACHEIV Post-operative metabolic
                    17013, --APACHEIV Post-operative musculoskeletal /skin
                    17014, --APACHEIV Post-operative neurologic
                    17015, --APACHEIV Post-operative respiratory
                    17016, --APACHEIV Post-operative transplant
                    17017 --APACHEIV Post-operative trauma

                ) THEN 1
                WHEN itemid = 18669 AND valueid BETWEEN 1 AND 26 THEN 1 --NICE APACHEII diagnosen
                WHEN itemid = 18671 AND valueid BETWEEN 222 AND 452 THEN 1 --NICE APACHEIV diagnosen
                ELSE 0
            END AS surgical,
            valueid as diagnosis_id,
            CASE 
                    WHEN itemid = 18671 THEN 'NICE APACHE IV'
                    WHEN itemid = 18669 THEN 'NICE APACHE II'
                    WHEN itemid BETWEEN 16998 AND 17017 THEN 'APACHE IV'
                    WHEN itemid BETWEEN 18589 AND 18602 THEN 'APACHE II'
                    WHEN itemid BETWEEN 13116 AND 13145 THEN 'Legacy ICU'
                    WHEN itemid BETWEEN 16642 AND 16673 THEN 'Legacy MCU'
            END AS diagnosis_type,
            ROW_NUMBER() OVER(PARTITION BY admissionid
            ORDER BY 
                CASE --prefer NICE > APACHE IV > II > D
                    WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                    WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                    WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                    WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                    WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                    WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
                END DESC,
                measuredat DESC) AS rownum
        FROM chunk
        WHERE itemid IN (
            -- Diagnosis - LEVEL 2
            --SURGICAL
            13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
            16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
            13117, --D_Thoraxchirurgie_Cardio anders
            16672, --DMC_Thoraxchirurgie_Cardio anders
            13118, --D_Thoraxchirurgie_Aorta chirurgie
            16670, --DMC_Thoraxchirurgie_Aorta chirurgie
            13119, --D_Thoraxchirurgie_Pulmonale chirurgie
            16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

            13141, --D_Algemene chirurgie_Algemeen   
            16642, --DMC_Algemene chirurgie_Algemeen
            13121, --D_Algemene chirurgie_Buikchirurgie
            16643, --DMC_Algemene chirurgie_Buikchirurgie
            13123, --D_Algemene chirurgie_Endocrinologische chirurgie
            16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
            13145, --D_Algemene chirurgie_KNO/Overige
            16645, --DMC_Algemene chirurgie_KNO/Overige
            13125, --D_Algemene chirurgie_Orthopedische chirurgie
            16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
            13122, --D_Algemene chirurgie_Transplantatie chirurgie
            16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
            13124, --D_Algemene chirurgie_Trauma
            16648, --DMC_Algemene chirurgie_Trauma
            13126, --D_Algemene chirurgie_Urogenitaal
            16649, --DMC_Algemene chirurgie_Urogenitaal
            13120, --D_Algemene chirurgie_Vaatchirurgie
            16650, --DMC_Algemene chirurgie_Vaatchirurgie

            13128, --D_Neurochirurgie _Vasculair chirurgisch
            16661, --DMC_Neurochirurgie _Vasculair chirurgisch
            13129, --D_Neurochirurgie _Tumor chirurgie
            16660, --DMC_Neurochirurgie _Tumor chirurgie
            13130, --D_Neurochirurgie_Overige
            16662, --DMC_Neurochirurgie_Overige

            18596, --Apache II Operatief  Gastr-intenstinaal
            18597, --Apache II Operatief Cardiovasculair
            18598, --Apache II Operatief Hematologisch
            18599, --Apache II Operatief Metabolisme
            18600, --Apache II Operatief Neurologisch
            18601, --Apache II Operatief Renaal
            18602, --Apache II Operatief Respiratoir

            17008, --APACHEIV Post-operative cardiovascular
            17009, --APACHEIV Post-operative gastro-intestinal
            17010, --APACHEIV Post-operative genitourinary
            17011, --APACHEIV Post-operative hematology
            17012, --APACHEIV Post-operative metabolic
            17013, --APACHEIV Post-operative musculoskeletal /skin
            17014, --APACHEIV Post-operative neurologic
            17015, --APACHEIV Post-operative respiratory
            17016, --APACHEIV Post-operative transplant
            17017, --APACHEIV Post-operative trauma

            --MEDICAL
            13133, --D_Interne Geneeskunde_Cardiovasculair
            16653, --DMC_Interne Geneeskunde_Cardiovasculair
            13134, --D_Interne Geneeskunde_Pulmonaal
            16658, --DMC_Interne Geneeskunde_Pulmonaal
            13135, --D_Interne Geneeskunde_Abdominaal
            16652, --DMC_Interne Geneeskunde_Abdominaal
            13136, --D_Interne Geneeskunde_Infectieziekten
            16655, --DMC_Interne Geneeskunde_Infectieziekten
            13137, --D_Interne Geneeskunde_Metabool
            16656, --DMC_Interne Geneeskunde_Metabool
            13138, --D_Interne Geneeskunde_Renaal
            16659, --DMC_Interne Geneeskunde_Renaal
            13139, --D_Interne Geneeskunde_Hematologisch
            16654, --DMC_Interne Geneeskunde_Hematologisch
            13140, --D_Interne Geneeskunde_Overige
            16657, --DMC_Interne Geneeskunde_Overige

            13131, --D_Neurologie_Vasculair neurologisch
            16664, --DMC_Neurologie_Vasculair neurologisch
            13132, --D_Neurologie_Overige
            16663, --DMC_Neurologie_Overige 
            13127, --D_KNO/Overige

            18589, --Apache II Non-Operatief Cardiovasculair
            18590, --Apache II Non-Operatief Gastro-intestinaal
            18591, --Apache II Non-Operatief Hematologisch
            18592, --Apache II Non-Operatief Metabolisme
            18593, --Apache II Non-Operatief Neurologisch
            18594, --Apache II Non-Operatief Renaal
            18595, --Apache II Non-Operatief Respiratoir

            16998, --APACHE IV Non-operative cardiovascular
            16999, --APACHE IV Non-operative Gastro-intestinal
            17000, --APACHE IV Non-operative genitourinary
            17001, --APACHEIV  Non-operative haematological
            17002, --APACHEIV  Non-operative metabolic
            17003, --APACHEIV Non-operative musculo-skeletal
            17004, --APACHEIV Non-operative neurologic
            17005, --APACHEIV Non-operative respiratory
            17006, --APACHEIV Non-operative transplant
            17007, --APACHEIV Non-operative trauma

            --NICE: surgical/medical combined in same parameter
            18669, --NICE APACHEII diagnosen
            18671 --NICE APACHEIV diagnosen
        )
    """
    
    diagnosis_chunk = ps.sqldf(diagnosis_query)
    
    if diagnosis is None:
        diagnosis = diagnosis_chunk
    else: 
        diagnosis = pd.concat([diagnosis,diagnosis_chunk])
        
diagnosis.to_csv("Data/Extracted/combined_diagnosis/diagnosis.csv",index=False)

In [ ]:
#Takes around 5 minutes
sepsis = None

for chunk in tqdm.tqdm(pd.read_csv(r"Data/listitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):

    sepsis_query = """
    SELECT
        admissionid,
        CASE valueid
            WHEN 1 THEN 1 --'Ja'
            WHEN 2 THEN 0 --'Nee'
        END as sepsis_at_admission,
        ROW_NUMBER() OVER(
            PARTITION BY 
                admissionid
            ORDER BY 
                measuredat DESC) AS rownum
    FROM chunk
    WHERE 
        itemid = 15808
    """
    sepsis_chunk = ps.sqldf(sepsis_query)
    
    if sepsis is None:
        sepsis = sepsis_chunk
    else: 
        sepsis = pd.concat([sepsis,sepsis_chunk])
        
sepsis.to_csv("Data/Extracted/combined_diagnosis/sepsis.csv",index=False)

In [ ]:
other_antibiotics = None
sepsis_antibiotics = None

#This takes around 2 minutes

#drugitems = pd.read_csv(r"Data/drugitems.csv",sep=',', encoding='latin-1')
for chunk in tqdm.tqdm(pd.read_csv(r"Data/drugitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):

    sepsis_antibiotics_query = """
    --non prophylactic antibiotics
        SELECT
            admissionid,
            CASE 
                WHEN COUNT(*) > 0 THEN 1
                ELSE 0
            END AS sepsis_antibiotics_bool,
            REPLACE(REPLACE(GROUP_CONCAT(DISTINCT REPLACE(item, ',', '*')), ',', ';'), '*', ',') AS sepsis_antibiotics_given -- SQLite version
        FROM chunk
        WHERE 
            itemid IN (
                6834, --Amikacine (Amukin)
                6847, --Amoxicilline (Clamoxyl/Flemoxin)
                6871, --Benzylpenicilline (Penicilline)
                6917, --Ceftazidim (Fortum)
                --6919, --Cefotaxim (Claforan) -> prophylaxis
                6948, --Ciprofloxacine (Ciproxin)
                6953, --Rifampicine (Rifadin)
                6958, --Clindamycine (Dalacin)
                7044, --Tobramycine (Obracin)
                --7064, --Vancomycine -> prophylaxis for valve surgery
                7123, --Imipenem (Tienam)
                7185, --Doxycycline (Vibramycine)
                --7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
                --7208, --Erythromycine (Erythrocine) -> often used for gastroparesis
                7227, --Flucloxacilline (Stafoxil/Floxapen)
                7231, --Fluconazol (Diflucan)
                7232, --Ganciclovir (Cymevene)
                7233, --Flucytosine (Ancotil)
                7235, --Gentamicine (Garamycin)
                7243, --Foscarnet trinatrium (Foscavir)
                7450, --Amfotericine B (Fungizone)
                --7504, --X nader te bepalen --non-stock medication
                8127, --Meropenem (Meronem)
                8229, --Myambutol (ethambutol)
                8374, --Kinine dihydrocloride
                --8375, --Immunoglobuline (Nanogam) -> not anbiotic
                --8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
                8547, --Voriconazol(VFEND)
                --9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
                9030, --Aztreonam (Azactam)
                9047, --Chlooramfenicol
                --9075, --Fusidinezuur (Fucidin) -> prophylaxis
                9128, --Piperacilline (Pipcil)
                9133, --Ceftriaxon (Rocephin)
                --9151, --Cefuroxim (Zinacef) -> often used for GI/transplant surgical prophylaxis
                --9152, --Cefazoline (Kefzol) -> prophylaxis for cardiac surgery
                9458, --Caspofungine
                9542, --Itraconazol (Trisporal)
                --9602, --Tetanusimmunoglobuline -> prophylaxis/not antibiotic
                12398, --Levofloxacine (Tavanic)
                12772, --Amfotericine B lipidencomplex  (Abelcet)
                15739, --Ecalta (Anidulafungine)
                16367, --Research Anidulafungin/placebo
                16368, --Research Caspofungin/placebo
                18675, --Amfotericine B in liposomen (Ambisome )
                19137, --Linezolid (Zyvoxid)
                19764, --Tigecycline (Tygacil)
                19773, --Daptomycine (Cubicin)
                20175 --Colistine
            )
            AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
        GROUP BY admissionid"""
    sepsis_antibiotics_chunk = ps.sqldf(sepsis_antibiotics_query)
    
    if sepsis_antibiotics is None:
        sepsis_antibiotics = sepsis_antibiotics_chunk
    else: 
        sepsis_antibiotics = pd.concat([sepsis_antibiotics,sepsis_antibiotics_chunk])
        
sepsis_antibiotics.to_csv("Data/Extracted/combined_diagnosis/sepsis_antibiotics.csv",index=False)        

In [ ]:
for chunk in tqdm.tqdm(pd.read_csv(r"Data/drugitems.csv",sep=',',
                             chunksize=1000000, 
                             iterator=True, encoding='latin-1'),ascii=True):

    other_antibiotics_query = """
     --'prophylactic' antibiotics that may be used for sepsis
        SELECT
            admissionid,
            CASE 
                WHEN COUNT(*) > 0 THEN 1
                ELSE 0
            END AS other_antibiotics_bool,
            REPLACE(REPLACE(GROUP_CONCAT(DISTINCT REPLACE(item, ',', '*')), ',', ';'), '*', ',') AS other_antibiotics_given -- SQLite version
        FROM chunk
        WHERE 
            itemid IN (
                7064, --Vancomycine -> prophylaxis for valve surgery
                7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
                8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
                9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
                9151, --Cefuroxim (Zinacef) -> often used for GI surgical prophylaxis
                9152 --Cefazoline (Kefzol) -> prophylaxis
            )
            AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
        GROUP BY admissionid    
        """
    other_antibiotics_chunk = ps.sqldf(other_antibiotics_query)
    
    if other_antibiotics is None:
        other_antibiotics = other_antibiotics_chunk
    else: 
        other_antibiotics = pd.concat([other_antibiotics,other_antibiotics_chunk])
    
other_antibiotics.to_csv("Data/Extracted/combined_diagnosis/other_antibiotics.csv",index=False)

In [ ]:
procedureorderitems = pd.read_csv(r"Data/procedureorderitems.csv",sep=',', encoding='latin-1')

cultures_query =""" 
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_cultures_bool,
        REPLACE(REPLACE(GROUP_CONCAT(DISTINCT REPLACE(item, ',', '*')), ',', ';'), '*', ',') AS sepsis_cultures_drawn
    FROM procedureorderitems
    WHERE 
        itemid IN (
        --8097, --Sputumkweek afnemen -> often used routinely
        --8418, --Urinekweek afnemen
        --8588, --MRSA kweken afnemen 
        9189, --Bloedkweken afnemen
        9190, --Cathetertipkweek afnemen
        --9191, --Drainvochtkweek afnemen
        --9192, --Faeceskweek afnemen -> Clostridium
        --9193, --X-Kweek nader te bepalen
        --9194, --Liquorkweek afnemen
        --9195, --Neuskweek afnemen
        --9197, --Perineumkweek afnemen -> often used routinely
        -9198, --Rectumkweek afnemen -> often used routinely
        9200, --Wondkweek afnemen
        9202, --Ascitesvochtkweek afnemen
        --9203, --Keelkweek afnemen -> often used routinely
        --9204, --SDD-kweken afnemen -> often used routinely
        9205 --Legionella sneltest (urine)
        --1302, --SDD Inventarisatiekweken afnemen -> often used routinely
        --19663, --Research Neuskweek COUrSe
        --19664, --Research Sputumkweek COUrSe
        )
        AND registeredat < 6*60*60*1000 --within 6 hours
    GROUP BY admissionid"""
cultures = ps.sqldf(cultures_query)

cultures.to_csv("Data/Extracted/combined_diagnosis/cultures.csv",index=False)
procedureorderitems = None #save RAM

In [ ]:
admissions = pd.read_csv(r"Data/admissions.csv",sep=',', encoding='latin-1')
diagnosis = pd.read_csv(r"Data/Extracted/combined_diagnosis/diagnosis.csv",sep=',')
diagnosis_subgroups = pd.read_csv(r"Data/Extracted/combined_diagnosis/diagnosis_subgroups.csv",sep=',')
diagnosis_groups = pd.read_csv(r"Data/Extracted/combined_diagnosis/diagnosis_groups.csv",sep=',')
sepsis = pd.read_csv(r"Data/Extracted/combined_diagnosis/sepsis.csv",sep=',')
sepsis_antibiotics = pd.read_csv(r"Data/Extracted/combined_diagnosis/sepsis_antibiotics.csv",sep=',')
other_antibiotics = pd.read_csv(r"Data/Extracted/combined_diagnosis/other_antibiotics.csv",sep=',')
cultures = pd.read_csv(r"Data/Extracted/combined_diagnosis/cultures.csv",sep=',')

combined_diagnosis_query = """
SELECT 
    admissions.* 
    , diagnosis_type
    , diagnosis, diagnosis_id
    , diagnosis_subgroup
    , diagnosis_subgroup_id
    , diagnosis_group
    , diagnosis_group_id
    , surgical
    , sepsis_at_admission
    , sepsis_antibiotics_bool
    , sepsis_antibiotics_given
    , other_antibiotics_bool
    , other_antibiotics_given
    , sepsis_cultures_bool
    , sepsis_cultures_drawn
FROM admissions
LEFT JOIN diagnosis on admissions.admissionid = diagnosis.admissionid
LEFT JOIN diagnosis_subgroups on admissions.admissionid = diagnosis_subgroups.admissionid
LEFT JOIN diagnosis_groups on admissions.admissionid = diagnosis_groups.admissionid
LEFT JOIN sepsis on admissions.admissionid = sepsis.admissionid
LEFT JOIN sepsis_antibiotics on admissions.admissionid = sepsis_antibiotics.admissionid
LEFT JOIN other_antibiotics on admissions.admissionid = other_antibiotics.admissionid
LEFT JOIN cultures on admissions.admissionid = cultures.admissionid
WHERE --only last updated record
    (diagnosis.rownum = 1 OR diagnosis.rownum IS NULL) AND 
    (diagnosis_subgroups.rownum = 1 OR diagnosis_subgroups.rownum IS NULL) AND
    (diagnosis_groups.rownum = 1 OR diagnosis_groups.rownum IS NULL) AND
    (sepsis.rownum = 1 OR sepsis.rownum IS NULL) 
"""
combined_diagnoses = ps.sqldf(combined_diagnosis_query)
combined_diagnoses.to_csv("Data/Extracted/combined_diagnosis.csv",index=False)

# Atrial Fibrillation DataFrame
This contains the query and any general preprocessing to make the DataFrame workable

In [4]:
afib_or = pd.read_csv(data_amsterdam_get_path+"Extracted/afib.csv")
afib_or.head(1000)

,Unnamed: 0,admissionid,itemid,item,valueid,value,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult
0,4614,6,6671,Hartritme,13,Atr fib,2340000,"5,460,000.00",ICV_IC-Verpleegkundig,5460000,ICV_IC-Verpleegkundig,0
1,4615,6,6671,Hartritme,13,Atr fib,5940000,"9,360,000.00",ICV_IC-Verpleegkundig,9360000,ICV_IC-Verpleegkundig,0
2,4616,6,6671,Hartritme,13,Atr fib,9540000,"12,180,000.00",ICV_IC-Verpleegkundig,12180000,ICV_IC-Verpleegkundig,0
3,4617,6,6671,Hartritme,13,Atr fib,13140000,"14,580,000.00",ICV_IC-Verpleegkundig,14580000,ICV_IC-Verpleegkundig,0
4,4618,6,6671,Hartritme,13,Atr fib,16740000,"18,120,000.00",ICV_IC-Verpleegkundig,18120000,ICV_IC-Verpleegkundig,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,227243,160,6671,Hartritme,13,Atr fib,750900000,"750,960,000.00",ICV_IC-Verpleegkundig,750960000,ICV_IC-Verpleegkundig,0
996,227244,160,6671,Hartritme,13,Atr fib,754500000,"755,160,000.00",ICV_IC-Verpleegkundig,755160000,ICV_IC-Verpleegkundig,0
997,227245,160,6671,Hartritme,13,Atr fib,758100000,"758,940,000.00",ICV_IC-Verpleegkundig,758940000,ICV_IC-Verpleegkundig,0
998,227246,160,6671,Hartritme,13,Atr fib,761700000,"761,760,000.00",ICV_IC-Verpleegkundig,761760000,ICV_IC-Verpleegkundig,0


In [5]:
afib = afib_or.copy(deep=True)
afib["measuredat_delta"] = pd.to_timedelta(afib_or.measuredat,'ms')
afib["date"] = (today+afib.measuredat_delta).dt.date

## Admission DataFrame
The SQL for combined diagnosis is copied from: [UMCdb 
Reason for admission](https://render.githubusercontent.com/view/ipynb?color_mode=auto&commit=9c66f2bb801266b00cf3b4aaa9d0afc580cd7253&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f416d7374657264616d554d432f416d7374657264616d554d4364622f396336366632626238303132363662303063663362346161613964306166633538306364373235332f636f6e63657074732f646961676e6f7369732f726561736f6e5f666f725f61646d697373696f6e2e6970796e62&nwo=AmsterdamUMC%2FAmsterdamUMCdb&path=concepts%2Fdiagnosis%2Freason_for_admission.ipynb&repository_id=213852696&repository_type=Repository#Sepsis)

In [6]:
combined_diagnoses = pd.read_csv(data_amsterdam_get_path+"Extracted/combined_diagnosis.csv")
combined_diagnoses

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,...,diagnosis_group,diagnosis_group_id,surgical,sepsis_at_admission,sepsis_antibiotics_bool,sepsis_antibiotics_given,other_antibiotics_bool,other_antibiotics_given,sepsis_cultures_bool,sepsis_cultures_drawn
0,0,0,1,IC,0,NaN,0,2003-2009,148800000,42,...,Thoraxchirurgie,1.00,1.00,NaN,NaN,NaN,1.00,Cefazoline (Kefzol),NaN,NaN
1,1,1,1,IC,0,NaN,0,2010-2016,96120000,26,...,Thoraxchirurgie,1.00,1.00,0.00,NaN,NaN,1.00,Vancomycine;Cefazoline (Kefzol),NaN,NaN
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,...,Post-operative cardiovascular,237.00,1.00,0.00,NaN,NaN,1.00,Vancomycine;Cefazoline (Kefzol),NaN,NaN
3,3,3,1,IC,0,NaN,0,2003-2009,84900000,23,...,NaN,NaN,NaN,NaN,NaN,NaN,1.00,Cefazoline (Kefzol),NaN,NaN
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,...,Post-operative cardiovascular,233.00,1.00,0.00,NaN,NaN,1.00,Cefazoline (Kefzol),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23104,20322,23548,1,IC&MC,1,NaN,0,2010-2016,949380000,264,...,Neurologie,4.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
23105,20323,23549,1,MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,56820000,16,...,Post-operative transplant,408.00,1.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
23106,20324,23550,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,69720000,20,...,Non-operative respiratory,162.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
23107,20325,23551,1,IC,1,NaN,0,2003-2009,165660000,46,...,Neurochirurgie,3.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
admissions_pd_or = pd.read_csv(data_amsterdam_get_path+"admissions.csv")
admissions_pd_or

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty
0,0,0,1,IC,0,NaN,0,2003-2009,148800000,42,16,Vrouw,80+,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie
1,1,1,1,IC,0,NaN,0,2010-2016,96120000,26,15,Man,60-69,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,15,Man,60-69,NaN,90-99,Anamnestisch,180-189,Anamnestisch,Cardiochirurgie
3,3,3,1,IC,0,NaN,0,2003-2009,84900000,23,14,Man,50-59,NaN,90-99,NaN,180-189,Gemeten,Cardiochirurgie
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,19,Man,70-79,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23101,20322,23548,1,IC&MC,1,NaN,0,2010-2016,949380000,264,45,Vrouw,40-49,NaN,70-79,Anamnestisch,180-189,Anamnestisch,Neurochirurgie
23102,20323,23549,1,MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,56820000,16,41,Vrouw,40-49,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Nefrologie
23103,20324,23550,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,69720000,20,Overleden,Man,80+,"-35,640,000.00",59-,Geschat,170-179,Geschat,Inwendig
23104,20325,23551,1,IC,1,NaN,0,2003-2009,165660000,46,Overleden,Vrouw,40-49,"115,500,000.00",60-69,Geschat,160-169,Geschat,Neurochirurgie


In [8]:
admissions_pd = admissions_pd_or.copy(deep=True)
admissions_pd = admissions_pd.fillna(0) # Fill not available values with a 0. 
admissions_pd["dateofdeath_delta"] = pd.to_timedelta(admissions_pd.dateofdeath,'ms')
admissions_pd["admittedat_delta"] = pd.to_timedelta(admissions_pd.admittedat,'ms')

#Convert the string categories into integer numbers. 
weight_group_numb = {'Group':['70-79', '60-69', '80-89', '59-', '90-99', '110+', '100-109', 0],'Number':[75,65,85,55,95,115,105,0]}
height_group_numb = {'Group':['180-189', '160-169', '170-179', '190+', '159-', 0],'Number':[185,165,175,195,155,0]}
age_group_numb = {'Group':['70-79', '60-69', '50-59', '18-39', '80+', '40-49'],'Number':[75,65,55,35,85,45]}

#Map each group to a number, e.g. '70-79':75 etc..
weight_group_numb = dict(zip(weight_group_numb['Group'],weight_group_numb["Number"]))
height_group_numb = dict(zip(height_group_numb['Group'],height_group_numb["Number"]))
age_group_numb = dict(zip(age_group_numb['Group'],age_group_numb["Number"]))

#If no gender was specified, the value is -1.
gender_category = {'Man':1,"Vrouw":0,"":-1}

#Replace the strings with numbers
admissions_pd["Height"]=admissions_pd["heightgroup"].replace(height_group_numb)
admissions_pd["Age"]=admissions_pd["agegroup"].replace(age_group_numb)
admissions_pd["Weight"]=admissions_pd["weightgroup"].replace(weight_group_numb)
admissions_pd["Gender_category"]=admissions_pd["gender"].replace(gender_category)

#Create a mortality column
admissions_pd["Mortality"]=0
admissions_pd.loc[admissions_pd.dateofdeath>0,"Mortality"]=1

#Add the sepsis definition to the admission_pd
re_sepsis_surg = r'sepsis|pneumoni|GI perforation|perforation/rupture|infection|abscess|GI Vascular ischemia|diverticular|appendectomy|peritonitis'
re_sepsis_med = r'sepsis|septic|infect|pneumoni|cholangitis|pancr|endocarditis|meningitis|GI perforation|abces|abscess|darm ischaemie|GI vascular|fasciitis|inflammatory|peritonitis'

sepsis = combined_diagnoses[
    (
        (
            #use reasons for admission
            #surgical admissions with sepsis
            (combined_diagnoses['surgical'] == 1) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
        ) | (
            #medical admissions with sepsis
            (combined_diagnoses['surgical'] == 0) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
        ) | (
            #uses documentation at admission form (Early Goal Directed Therapy)
            (combined_diagnoses['sepsis_at_admission'] == 1)
        ) | (
            #uses administered (therapeutic) antibiotics for determining sepsis
            (combined_diagnoses['sepsis_antibiotics_bool'] == 1)
        ) | (
            #uses combination of administered antibiotics (that sometimes are used as prophylaxis) AND 
            #drawn cultures for determining sepsis
            (combined_diagnoses['other_antibiotics_bool'] == 1) 
            & (combined_diagnoses['sepsis_cultures_bool'] == 1)
        )
    ) & ~(combined_diagnoses['sepsis_at_admission'] == 0) 
    #exclude all diagnoses where explicitly 'no sepsis' was documented
]
admissions_pd["sepsis_bool"]=0
admissions_pd.loc[admissions_pd.admissionid.isin(sepsis.admissionid),"sepsis_bool"]=1

##Add the neurology diagnosis to the admission pd
re_neuro = '(?<!see )(insult|seizure|CVA|observatie neurologische status|intracerebraal haematoom|intracranial|intracerebr|subdur|subarachno|epidur|coma|neurologisch|neurologic|CZS|S\.A\.B\.|neurologie|hoofdtrauma|head|neuro|muscula|spinal|meningitis|encephalitis|myasthenia|vaatspasme protocol|Guillian-Barre|encephalopath|musculoskeletal)(?!-see Neurological System)'
re_trauma_surg = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'

neuro = combined_diagnoses[
    (combined_diagnoses['surgical'] == 0) 
    & (combined_diagnoses['diagnosis'].str.contains(re_neuro, na=False, flags=re.IGNORECASE))
    #exclude trauma cases
    & ~(combined_diagnoses['diagnosis'].str.contains(re_trauma_surg, na=False, flags=re.IGNORECASE))
]
admissions_pd["neuro_bool"]=0
admissions_pd.loc[admissions_pd.admissionid.isin(neuro.admissionid),"neuro_bool"]=1

## Add the cardiac surgery to the admission pd
re_cardiosurg = r'(CABG|AVR|hartchirurgie|heart surgery|Chron. cardiovasculaire ziekte|hartkleppen|cardiovascula|MVP|MVR|mitral|tricuspid|pericard|aortic.*valve|lobectom|segment|thorax|Bentall|aorta-ascendens|aorta-boog|aorta-wortel|aorta-descendens|lung|pneumectomie|bullectom|respiratoir neoplasm|thoracoscop|thoracotom(y|ie)|respirato|vrije wand ruptuur|VSR|ASD|pleurectom|intracardiac|aneurysmectom|congenital defect repair)(?! for esophag)'

cardiosurg = combined_diagnoses[
    (combined_diagnoses['surgical'] == 1) 
    & (combined_diagnoses['diagnosis'].str.contains(re_cardiosurg, na=False, flags=re.IGNORECASE))]
admissions_pd["cardiac_surg_bool"]=0
admissions_pd.loc[admissions_pd.admissionid.isin(cardiosurg.admissionid),"cardiac_surg_bool"]=1

re_cardiosurg_new = (
    r'(CABG|AVR|hartchirurgie|Chron. cardiovasculaire ziekte|hartkleppen|'
    r'MVP|MVR|mitral|tricuspid|pericard|aortic.*valve|Bentall|'
    r'vrije wand ruptuur|VSR|ASD|intracardiac|congenital defect repair)(?! for esophag)'
)
cardiosurg_new = combined_diagnoses[(combined_diagnoses['surgical'] == 1) & (combined_diagnoses['diagnosis'].str.contains(re_cardiosurg_new, na=False, flags=re.IGNORECASE))]

admissions_pd["cardio_surgery_new"] = 0
admissions_pd.loc[admissions_pd.admissionid.isin(cardiosurg_new.admissionid),"cardio_surgery_new"] = 1

admissions_pd.head(1000)

This pattern has match groups. To actually get the groups, use str.extract.
This pattern has match groups. To actually get the groups, use str.extract.
This pattern has match groups. To actually get the groups, use str.extract.
This pattern has match groups. To actually get the groups, use str.extract.


,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,...,admittedat_delta,Height,Age,Weight,Gender_category,Mortality,sepsis_bool,neuro_bool,cardiac_surg_bool,cardio_surgery_new
0,0,0,1,IC,0,0,0,2003-2009,148800000,42,...,0 days,165,85,65,0,0,0,0,1,1
1,1,1,1,IC,0,0,0,2010-2016,96120000,26,...,0 days,175,65,75,1,0,0,0,1,1
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,...,0 days,185,65,95,1,0,0,0,1,1
3,3,3,1,IC,0,0,0,2003-2009,84900000,23,...,0 days,185,55,95,1,0,0,0,0,0
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,...,0 days,175,75,75,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,883,1014,1,IC,0,0,0,2003-2009,85080000,23,...,0 days,175,45,75,1,0,0,0,1,1
996,884,1015,1,MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,69900000,20,...,0 days,165,55,75,0,0,0,0,0,0
997,885,1016,1,IC,1,0,0,2010-2016,704580000,196,...,0 days,185,75,105,1,1,0,0,0,0
998,886,1017,1,IC,0,0,0,2003-2009,86340000,24,...,0 days,175,75,75,1,0,0,0,0,0


## Atrial Fibrillation Preprocessing

In [9]:
afib_merged = afib.merge(admissions_pd,how='left',on='admissionid')

# Calculate the actual time to the current admission for the AF diagnosis. If a value is in the admittedat, then this admission is not the first admission of the patient. 
afib_merged.measuredat = afib_merged.measuredat.values-afib_merged.admittedat.values

In [10]:
# plt.figure(figsize=(15,9))
# plt.hist((afib_merged.registeredat-afib_merged.measuredat)/1000/60,bins=np.arange(-20,176,5),color="#e898ac",ec='white')
# plt.ylabel("Count")
# plt.xlim([-20,170])
# plt.xticks(np.arange(-20,176,10))
# plt.grid()
# plt.xlabel("Minutes between registering and measuring AF")
# if present:
#     ax = plt.gca()
#     ax.patch.set_alpha(0)

In [11]:
new_onset_interval = 1 #the amount of time after admission, where the occurence of AF is new onset and not preadmission

sorted_afib_merged = afib_merged.sort_values(["admissionid","measuredat"]).drop_duplicates("admissionid",keep='first')
sorted_afib_merged_new_onset = sorted_afib_merged[(sorted_afib_merged.measuredat/1000/60/60>new_onset_interval)
                                                  &(~sorted_afib_merged.admissionid.isin(sorted_afib_merged[sorted_afib_merged.measuredat/1000/60/60<=new_onset_interval].admissionid.values))] #select when AF is new-onset

In [12]:
admissions_pd["AF"]=0
admissions_pd.loc[admissions_pd.admissionid.isin(afib.admissionid.values),"AF"]=1
admissions_pd["new_onset_AF"]=0
admissions_pd.loc[admissions_pd.admissionid.isin(sorted_afib_merged_new_onset.admissionid.values),"new_onset_AF"]=1
admissions_pd["Preadmission_AF"]=admissions_pd["AF"].values*(1-admissions_pd["new_onset_AF"].values)
admissions_pd["AF_measuredat"]=admissions_pd.merge(afib_merged[["admissionid","measuredat"]].sort_values(["admissionid","measuredat"]).drop_duplicates("admissionid",keep='first'),how='left',on='admissionid').measuredat.values

admissions_pd.head(1000)

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,...,Gender_category,Mortality,sepsis_bool,neuro_bool,cardiac_surg_bool,cardio_surgery_new,AF,new_onset_AF,Preadmission_AF,AF_measuredat
0,0,0,1,IC,0,0,0,2003-2009,148800000,42,...,0,0,0,0,1,1,0,0,0,NaN
1,1,1,1,IC,0,0,0,2010-2016,96120000,26,...,1,0,0,0,1,1,0,0,0,NaN
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,...,1,0,0,0,1,1,0,0,0,NaN
3,3,3,1,IC,0,0,0,2003-2009,84900000,23,...,1,0,0,0,0,0,0,0,0,NaN
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,...,1,0,0,0,1,1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,883,1014,1,IC,0,0,0,2003-2009,85080000,23,...,1,0,0,0,1,1,0,0,0,NaN
996,884,1015,1,MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,69900000,20,...,0,0,0,0,0,0,0,0,0,NaN
997,885,1016,1,IC,1,0,0,2010-2016,704580000,196,...,1,1,0,0,0,0,1,1,0,"3,780,000.00"
998,886,1017,1,IC,0,0,0,2003-2009,86340000,24,...,1,0,0,0,0,0,0,0,0,NaN


In [13]:
admissions_pd.describe()

,patientid,admissionid,admissioncount,urgency,admittedat,dischargedat,lengthofstay,dateofdeath,dateofdeath_delta,admittedat_delta,...,Gender_category,Mortality,sepsis_bool,neuro_bool,cardiac_surg_bool,cardio_surgery_new,AF,new_onset_AF,Preadmission_AF,AF_measuredat
count,"23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00",23106,23106,...,"23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","23,106.00","4,086.00"
mean,"10,161.07","11,769.03",1.17,0.27,"3,953,680,851.73","4,353,618,488.70",111.10,"20,323,668,075.82",235 days 05:27:48.075824464,45 days 18:14:40.851726824,...,0.64,0.32,0.14,0.03,0.26,0.23,0.18,0.13,0.05,"166,126,901.62"
std,"5,857.73","6,790.32",0.50,0.44,"23,867,461,623.11","23,894,276,883.42",233.11,"58,344,657,832.83",675 days 06:50:57.832830416,276 days 05:51:01.623112572,...,0.48,0.47,0.34,0.16,0.44,0.42,0.38,0.34,0.21,"397,675,126.32"
min,0.00,0.00,1.00,0.00,0.00,0.00,0.00,"-138,294,360,000.00",-1601 days +08:54:00,0 days 00:00:00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-3,540,000.00"
25%,"5,101.50","5,886.50",1.00,0.00,0.00,"75,120,000.00",20.00,0.00,0 days 00:00:00,0 days 00:00:00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,955,000.00"
50%,"10,156.50","11,765.50",1.00,0.00,0.00,"112,200,000.00",26.00,0.00,0 days 00:00:00,0 days 00:00:00,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"43,680,000.00"
75%,"15,237.00","17,647.75",1.00,1.00,0.00,"550,875,000.00",88.00,"999,705,000.00",11 days 13:41:45,0 days 00:00:00,...,1.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,"175,845,000.00"
max,"20,326.00","23,552.00",8.00,1.00,"391,652,460,000.00","391,668,660,000.00","5,697.00","459,590,400,000.00",5319 days 08:00:00,4533 days 00:21:00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,"7,792,800,000.00"


# AF

## Data preparation

In [14]:
#This takes eight minutes
numeric_items_pd = pd.read_csv(data_amsterdam_get_path+"Extracted/numeric_items_af_features_new.csv")
numeric_items_pd.head(1000)

,admissionid,value,measuredat,itemid
0,0,83.00,20520000,6640
1,0,84.00,22320000,6640
2,0,82.00,25920000,6640
3,0,78.00,29520000,6640
4,0,73.00,33120000,6640
...,...,...,...,...
995,4,104.00,58500000,6640
996,4,103.00,58560000,6640
997,4,104.00,58620000,6640
998,4,105.00,58680000,6640


In [15]:
#This takes arounde one minute
drug_items_pd = pd.read_csv(data_amsterdam_get_path+"Extracted/drug_items_features_new.csv")

In [16]:
numeric_fluidbalance_pd = pd.read_csv(data_amsterdam_get_path+"Extracted/fluidbalance_numeric.csv")

In [17]:
drugitems_fluidbalance_pd = pd.read_csv(data_amsterdam_get_path+"Extracted/drugitems_fluidbalance.csv")

In [18]:
hours_to_first_AF = 12
numeric_items_pd["measuredat_hour"] = np.floor(numeric_items_pd.measuredat/1000/60/60) #per hour
numeric_items_pd["measuredat_min"] = np.floor(numeric_items_pd.measuredat/1000/60) #per minute
numeric_items_pd["measuredat_hours_to_first_AF"] = np.floor(numeric_items_pd.measuredat/1000/60/60/hours_to_first_AF) #per minute

#### NOR visualisation

In [ ]:
NOR_pd = drug_items_pd[drug_items_pd.itemid==7229]

time_shift = 0
margin_time = 0
hours_to_first_AF = 12

NOR_pd["administered_rate_min"] = NOR_pd["administered"]/NOR_pd["duration"]
NOR_pd["measuredat_min_avg"] = (NOR_pd["start"]+NOR_pd["stop"])/(2*(1000*60))

only_data_of_first_hours = False #Use only data of the first X hours of admission
include_AF_patients_without_AF_at_sample = True #Include moments in time of AF patients before the AF diagnosis in the dataset.

np.random.seed(42)

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=hours_to_first_AF)].copy(deep=True)
admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: 1000*60*60*np.random.randint(hours_to_first_AF,row['lengthofstay']+1) if pd.isnull(row['AF_measuredat']) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.

#AF patients
first_AFs_pd_timed = admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat/1000/60/60>=hours_to_first_AF)].reset_index(drop=True)
#indicate whether the patient is an AF patient or not, this is more for performance analysis of the model
first_AFs_pd_timed["AF_orig"] = 1

#No AF patients
#no_AF_same_length = admission_pd_min_hour[(admission_pd_min_hour.AF==0)&(admission_pd_min_hour.lengthofstay>=hours_to_first_AF)].sample(len(first_AFs_pd_timed),random_state=1).reset_index(drop=True)
no_AF_same_length = admission_pd_min_hour[(admission_pd_min_hour.AF==0)&(admission_pd_min_hour.lengthofstay>=hours_to_first_AF)].reset_index(drop=True)
no_AF_same_length["AF_orig"] = 0

temp_pd = no_AF_same_length

#AF_admission_dataset = first_AFs_pd_timed.append(no_AF_same_length).reset_index()
AF_admission_dataset = first_AFs_pd_timed.append(temp_pd).reset_index(drop=True)

hours_to_first_AF = 12

if only_data_of_first_hours:
    AF_admission_dataset["AF_measuredat"] = 1000*60*60*hours_to_first_AF #only take data of the first 48 hours

drug_items_patients = (NOR_pd[NOR_pd.admissionid.isin(AF_admission_dataset.admissionid)]).merge(AF_admission_dataset[["admissionid","AF_measuredat","AF"]],how='left',on='admissionid')
drug_items_patients["time_stop_to_AF"]=(drug_items_patients.AF_measuredat.values-1000*60*60*margin_time) - drug_items_patients.stop.values #add one hour to AF extra
drug_items_patients["time_start_to_AF"]=(drug_items_patients.AF_measuredat.values-1000*60*60*margin_time) - drug_items_patients.start.values #add one hour to AF extra

drug_items_patients = drug_items_patients[((drug_items_patients.time_stop_to_AF > (time_shift-margin_time)*1000*60*60)) & (drug_items_patients.time_stop_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                          ((drug_items_patients.time_start_to_AF > (time_shift-margin_time)*1000*60*60)) & (drug_items_patients.time_start_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                          ((drug_items_patients.time_start_to_AF <= (time_shift-margin_time)*1000*60*60)) & (drug_items_patients.time_stop_to_AF/1000/60/60 >= (time_shift+hours_to_first_AF-margin_time-1 ))]

for itemid_loop in NOR_pd.itemid.unique():
    drug_items_patients.itemid = drug_items_patients.itemid.replace(itemid_loop,dictionary[dictionary.itemid==itemid_loop].item.values[0])

In [ ]:
print(str(np.round(len(drug_items_patients[drug_items_patients.AF==1].admissionid.unique())/len(AF_admission_dataset[AF_admission_dataset.AF==1].admissionid.unique())*100,2))+" % of AF patients got NOR")
print(str(np.round(len(drug_items_patients[drug_items_patients.AF==0].admissionid.unique())/len(AF_admission_dataset[AF_admission_dataset.AF==0].admissionid.unique())*100,2))+" % of no AF patients got NOR")
print("So AF patients got "+str(np.round(np.round(len(drug_items_patients[drug_items_patients.AF==1].admissionid.unique())
                                      /len(AF_admission_dataset[AF_admission_dataset.AF==1].admissionid.unique())*100,2)
                             /np.round(len(drug_items_patients[drug_items_patients.AF==0].admissionid.unique())
                                           /len(AF_admission_dataset[AF_admission_dataset.AF==0].admissionid.unique())*100,2),2))+" times more often at least once NOR during admission")
print("")
print(str(np.round(len(drug_items_patients[drug_items_patients.AF==1].admissionid.unique())/len(drug_items_patients.admissionid.unique())*100,2))+" % of NOR patients had AF")
print(str(np.round(len(drug_items_patients[drug_items_patients.AF==0].admissionid.unique())/len(drug_items_patients.admissionid.unique())*100,2))+" % of NOR patients had no AF")
print(str(np.round(len(AF_admission_dataset[AF_admission_dataset.AF==1].admissionid.unique())/len(AF_admission_dataset.admissionid.unique())*100,2))+" % of ALL patients had AF")
print(str(np.round(len(drug_items_patients.admissionid.unique())/len(AF_admission_dataset.admissionid.unique())*100,2))+" % of ALL patients received NOR")

In [ ]:
min_plot = 0
max_plot = 50
stride = 5

fig, axs = plt.subplots(2,2,figsize=(16,12))

stop_arr_mins = drug_items_patients[drug_items_patients.AF==1][["admissionid","itemid","time_start_to_AF"]].groupby(["admissionid","itemid"]).agg({'max'}).reset_index()
axs[0,0].hist(np.clip(stop_arr_mins.time_start_to_AF["max"]/1000/60/60,min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),density=True,label="AF")
stop_arr_mins = drug_items_patients[drug_items_patients.AF==0][["admissionid","itemid","time_start_to_AF"]].groupby(["admissionid","itemid"]).agg({'max'}).reset_index()
axs[0,0].hist(np.clip(stop_arr_mins.time_start_to_AF["max"]/1000/60/60,min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),alpha=0.5,density=True,label="NO AF")
axs[0,0].legend()
axs[0,0].set_xlabel("Hours")
axs[0,0].set_title("Earliest start of NOR administration")


min_plot = -10
max_plot = 30
stride = 2

stop_arr_mins = drug_items_patients[drug_items_patients.AF==1][["admissionid","itemid","time_stop_to_AF"]].groupby(["admissionid","itemid"]).agg({'min'}).reset_index()
axs[1,0].hist(np.clip(stop_arr_mins.time_stop_to_AF["min"]/1000/60/60,min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),density=True,label="AF")
stop_arr_mins = drug_items_patients[drug_items_patients.AF==0][["admissionid","itemid","time_stop_to_AF"]].groupby(["admissionid","itemid"]).agg({'min'}).reset_index()
axs[1,0].hist(np.clip(stop_arr_mins.time_stop_to_AF["min"]/1000/60/60,min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),alpha=0.5,density=True,label="NO AF")
axs[1,0].legend()
axs[1,0].set_xlabel("Hours")
axs[1,0].set_title("Latest stop of NOR administration")

min_plot = 0
max_plot = 50
stride = 2

stop_arr_mins = drug_items_patients[drug_items_patients.AF==1][["admissionid","itemid","duration","AF_measuredat"]].groupby(["admissionid","itemid","AF_measuredat"]).agg({'sum'}).reset_index()
stop_arr_mins.duration = stop_arr_mins.duration/60 
axs[0,1].hist(np.clip(stop_arr_mins.duration["sum"],min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),density=True,label="AF")

stop_arr_mins = drug_items_patients[drug_items_patients.AF==0][["admissionid","itemid","duration","AF_measuredat"]].groupby(["admissionid","itemid","AF_measuredat"]).agg({'sum'}).reset_index()
stop_arr_mins.duration = stop_arr_mins.duration/60 
axs[0,1].hist(np.clip(stop_arr_mins.duration["sum"],min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),alpha=0.5,density=True,label="NO AF")
axs[0,1].legend()
axs[0,1].set_xlabel("Hours")
axs[0,1].set_title("Cumulative duration of NOR administration to measurement point")

min_plot = 0
max_plot = 1.05
stride = 0.05

stop_arr_mins_AF = drug_items_patients[drug_items_patients.AF==1][["admissionid","itemid","duration","AF_measuredat"]].groupby(["admissionid","itemid","AF_measuredat"]).agg({'sum'}).reset_index()
stop_arr_mins_AF.AF_measuredat = stop_arr_mins_AF.AF_measuredat/1000/60/60
stop_arr_mins_AF.duration = stop_arr_mins_AF.duration/60 
stop_arr_mins_AF["hours_to_first_AF"]=hours_to_first_AF

print("AF relative average NOR duration: "+str(np.mean(stop_arr_mins_AF[["duration","AF_measuredat","hours_to_first_AF"]].min(axis=1)/(stop_arr_mins_AF[["AF_measuredat","hours_to_first_AF"]].min(axis=1)))*100))

axs[1,1].hist(np.clip(stop_arr_mins_AF[["duration","AF_measuredat"]].min(axis=1)/(stop_arr_mins_AF.AF_measuredat),min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),density=True,label="AF")

stop_arr_mins_NO_AF = drug_items_patients[drug_items_patients.AF==0][["admissionid","itemid","duration","AF_measuredat"]].groupby(["admissionid","itemid","AF_measuredat"]).agg({'sum'}).reset_index()
stop_arr_mins_NO_AF.AF_measuredat = stop_arr_mins_NO_AF.AF_measuredat/1000/60/60
stop_arr_mins_NO_AF.duration = stop_arr_mins_NO_AF.duration/60 
stop_arr_mins_NO_AF["hours_to_first_AF"]=hours_to_first_AF
print("NO-AF relative average NOR duration: "+str(np.mean(stop_arr_mins_NO_AF[["duration","AF_measuredat","hours_to_first_AF"]].min(axis=1)/(stop_arr_mins_NO_AF[["AF_measuredat","hours_to_first_AF"]].min(axis=1)))*100))

print("P-value difference between both distributions: "+str(ttest_ind(stop_arr_mins_NO_AF[["duration","AF_measuredat","hours_to_first_AF"]].min(axis=1)/(stop_arr_mins_NO_AF[["AF_measuredat","hours_to_first_AF"]].min(axis=1)),stop_arr_mins_AF[["duration","AF_measuredat","hours_to_first_AF"]].min(axis=1)/(stop_arr_mins_AF[["AF_measuredat","hours_to_first_AF"]].min(axis=1)))[1]))

axs[1,1].hist(np.clip(stop_arr_mins_NO_AF[["duration","AF_measuredat"]].min(axis=1)/(stop_arr_mins_NO_AF.AF_measuredat),min_plot,max_plot-stride),bins=np.arange(min_plot,max_plot,stride),alpha=0.5,density=True,label="NO AF")
axs[1,1].legend()
axs[1,1].set_xlabel("Hours")
axs[1,1].set_title("Cumulative relative duration of NOR administration to measurement point")
plt.plot()

In [ ]:
drug_items_patients.loc[drug_items_patients["time_start_to_AF"]>60*60*1000*hours_to_first_AF,"time_start_to_AF"]=hours_to_first_AF*60*60*1000
drug_items_patients.loc[drug_items_patients["time_stop_to_AF"]<0,"time_stop_to_AF"]=0
drug_items_patients.loc[:,"duration"]=(drug_items_patients.time_start_to_AF.values-drug_items_patients.time_stop_to_AF.values)/1000/60
drug_items_patients["solutionadministered"]=drug_items_patients.rate*(drug_items_patients.duration/60)

stop_arr_mins_AF = drug_items_patients[drug_items_patients.AF==1][["admissionid","itemid","duration","AF_measuredat","solutionadministered"]].groupby(["admissionid","itemid","AF_measuredat"]).agg({'sum'}).reset_index()
stop_arr_mins_AF.AF_measuredat = stop_arr_mins_AF.AF_measuredat/1000/60/60
stop_arr_mins_AF.duration = stop_arr_mins_AF.duration/60 
stop_arr_mins_AF["hours_to_first_AF"]=hours_to_first_AF

print("AF relative average NOR dose rate for "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(stop_arr_mins_AF["solutionadministered"]["sum"]/(stop_arr_mins_AF[["AF_measuredat","hours_to_first_AF","duration"]].min(axis=1)))))

stop_arr_mins_NO_AF = drug_items_patients[drug_items_patients.AF==0][["admissionid","itemid","duration","AF_measuredat","solutionadministered"]].groupby(["admissionid","itemid","AF_measuredat"]).agg({'sum'}).reset_index()
stop_arr_mins_NO_AF.AF_measuredat = stop_arr_mins_NO_AF.AF_measuredat/1000/60/60
stop_arr_mins_NO_AF.duration = stop_arr_mins_NO_AF.duration/60 
stop_arr_mins_NO_AF["hours_to_first_AF"]=hours_to_first_AF
print("NO-AF relative average NOR dose rate for "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(stop_arr_mins_NO_AF["solutionadministered"]["sum"]/(stop_arr_mins_NO_AF[["AF_measuredat","hours_to_first_AF","duration"]].min(axis=1)))))
print("P-value difference between both distributions: "+str(ttest_ind(stop_arr_mins_NO_AF["solutionadministered"]["sum"]/(stop_arr_mins_NO_AF[["AF_measuredat","hours_to_first_AF","duration"]].min(axis=1)),stop_arr_mins_AF["solutionadministered"]["sum"]/(stop_arr_mins_AF[["AF_measuredat","hours_to_first_AF","duration"]].min(axis=1)))[1]))

In [ ]:
# SLOPE for rate administered min
drug_items_patients["administered_rate_hour"] = drug_items_patients["administered"]/(drug_items_patients["duration"]/60)
drug_items_patients["measuredat_hour_avg"] = (drug_items_patients["start"]+drug_items_patients["stop"])/(2*(1000*60*60))

drug_items_patients["cont"]=0 #is 0 if there is either 0 minutes between previous and current infusion or is 0 if it is the first infusion
drug_items_patients.loc[:,"cont"]=np.append(0,drug_items_patients.start.values[1:]-drug_items_patients.stop.values[:-1])# subtract the current start from the previous stop
drug_items_patients.loc[drug_items_patients.admissionid.diff()!=0,"cont"]=0
drug_items_patients = drug_items_patients.reset_index(drop=True)
drug_items_patients = drug_items_patients.reset_index()
drug_items_patients = drug_items_patients.rename(columns={'index':"or_idx"})

copy_arr = drug_items_patients[drug_items_patients.cont!=0].or_idx.values
i = 0
for el in copy_arr:
    if i%50 == 0:
        print("\rIteration: "+str(i)+"/"+str(len(copy_arr)),end="")
    i = i + 1
    replace_idx = drug_items_patients[drug_items_patients.or_idx==el].index[0]
    line = drug_items_patients.iloc[[replace_idx]].copy(deep=True)
    line.start = drug_items_patients.iloc[[replace_idx]].start-drug_items_patients.iloc[[replace_idx]].cont
    line.stop = drug_items_patients.iloc[[replace_idx]].start
    line.administered_rate_hour = 0
    drug_items_patients = pd.concat([drug_items_patients.iloc[:replace_idx], line, drug_items_patients.iloc[replace_idx:]]).reset_index(drop=True)
print("\r Done!                  ",end="")
drug_items_patients["measuredat_hour_avg"] = (drug_items_patients["start"]+drug_items_patients["stop"])/(2*(1000*60*60))

In [ ]:
drug_items_patients_slope_AF = drug_items_patients[drug_items_patients.AF==1][["admissionid","itemid","measuredat_hour_avg","administered_rate_hour"]].groupby(["admissionid","itemid"]).apply(lambda x:linregress(x.measuredat_hour_avg,x.administered_rate_hour)[0]).reset_index()
drug_items_patients_slope_AF.columns = [str(col) for col in drug_items_patients_slope_AF.columns.values]
drug_items_patients_slope_AF = drug_items_patients_slope_AF = drug_items_patients_slope_AF.rename(columns={'0':"slope"})
print("Average slope (mg/hour) of AF patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(drug_items_patients_slope_AF[~((drug_items_patients_slope_AF.slope.isna()) | (drug_items_patients_slope_AF.slope==0))].slope))+" mg/Hour")
plt.hist(np.clip(drug_items_patients_slope_AF[~((drug_items_patients_slope_AF.slope.isna()) | (drug_items_patients_slope_AF.slope==0))].slope,-0.1,0.08),bins=np.arange(-0.1,0.1,0.01),density=True,label="AF")

drug_items_patients_slope_NO_AF = drug_items_patients[drug_items_patients.AF==0][["admissionid","itemid","measuredat_hour_avg","administered_rate_hour"]].groupby(["admissionid","itemid"]).apply(lambda x:linregress(x.measuredat_hour_avg,x.administered_rate_hour)[0]).reset_index()
drug_items_patients_slope_NO_AF.columns = [str(col) for col in drug_items_patients_slope_NO_AF.columns.values]
drug_items_patients_slope_NO_AF = drug_items_patients_slope_NO_AF.rename(columns={'0':"slope"})
print("Average slope (mg/hour) of NO-AF patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(drug_items_patients_slope_NO_AF[~((drug_items_patients_slope_NO_AF.slope.isna()) | (drug_items_patients_slope_NO_AF.slope==0))].slope))+" mg/Hour")
plt.hist(np.clip(drug_items_patients_slope_NO_AF[~((drug_items_patients_slope_NO_AF.slope.isna()) | (drug_items_patients_slope_NO_AF.slope==0))].slope,-0.1,0.08),bins=np.arange(-0.1,0.1,0.01),alpha=0.5,density=True,label="NO AF")
plt.legend()
print("P-value difference between both distributions: "+str(ttest_ind(drug_items_patients_slope_NO_AF[~((drug_items_patients_slope_NO_AF.slope.isna()) | (drug_items_patients_slope_NO_AF.slope==0))].slope,drug_items_patients_slope_AF[~((drug_items_patients_slope_AF.slope.isna()) | (drug_items_patients_slope_AF.slope==0))].slope)[1]))

#### PEEP visualization

In [ ]:
PEEP_pd = pd.read_csv(data_amsterdam_get_path+"Extracted/numericitems/12284.csv")

In [ ]:
time_shift = 0
margin_time = 0
hours_to_first_AF = 12

PEEP_pd["measuredat_hour"] = np.floor(PEEP_pd.measuredat/1000/60/60) #per hour
PEEP_pd["measuredat_min"] = np.floor(PEEP_pd.measuredat/1000/60) #per minute

In [ ]:
only_data_of_first_hours = False #Use only data of the first X hours of admission
include_AF_patients_without_AF_at_sample = True #Include moments in time of AF patients before the AF diagnosis in the dataset.

np.random.seed(42)

hours_to_first_AF = 12

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=hours_to_first_AF)].copy(deep=True)
admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: 1000*60*60*np.random.randint(hours_to_first_AF,row['lengthofstay']+1) if pd.isnull(row['AF_measuredat']) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.

#AF patients
first_AFs_pd_timed = admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat/1000/60/60>=hours_to_first_AF)].reset_index(drop=True)
#indicate whether the patient is an AF patient or not, this is more for performance analysis of the model
first_AFs_pd_timed["AF_orig"] = 1

#No AF patients
#no_AF_same_length = admission_pd_min_hour[(admission_pd_min_hour.AF==0)&(admission_pd_min_hour.lengthofstay>=hours_to_first_AF)].sample(len(first_AFs_pd_timed),random_state=1).reset_index(drop=True)
no_AF_same_length = admission_pd_min_hour[(admission_pd_min_hour.AF==0)&(admission_pd_min_hour.lengthofstay>=hours_to_first_AF)].reset_index(drop=True)
no_AF_same_length["AF_orig"] = 0

temp_pd = no_AF_same_length

#AF_admission_dataset = first_AFs_pd_timed.append(no_AF_same_length).reset_index()
AF_admission_dataset = first_AFs_pd_timed.append(temp_pd).reset_index(drop=True)

if only_data_of_first_hours:
    AF_admission_dataset["AF_measuredat"] = 1000*60*60*hours_to_first_AF #only take data of the first 48 hours

hours_to_first_AF = 12

numeric_pd_patients = (PEEP_pd[PEEP_pd.admissionid.isin(AF_admission_dataset.admissionid)]).merge(AF_admission_dataset[["admissionid","AF_measuredat","AF"]],how='left',on='admissionid')
numeric_pd_patients["time_to_AF"]=(numeric_pd_patients.AF_measuredat.values-1000*60*60*margin_time) - numeric_pd_patients.measuredat.values #add one margin_time to AF extra
numeric_pd_patients = numeric_pd_patients[(numeric_pd_patients.time_to_AF > (time_shift-margin_time)*1000*60*60) & (numeric_pd_patients.time_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1) )]

for itemid_loop in PEEP_pd.itemid.unique():
    numeric_pd_patients.itemid = numeric_pd_patients.itemid.replace(itemid_loop,dictionary[dictionary.itemid==itemid_loop].item.values[0])

In [ ]:
print(str(hours_to_first_AF)+" Hours before measurement")
print(30*"-")
peep_setting = 4
print(str(np.round(len(numeric_pd_patients[(numeric_pd_patients.AF==1)&(numeric_pd_patients.value>peep_setting)].admissionid.unique())/len(AF_admission_dataset[AF_admission_dataset.AF==1].admissionid.unique())*100,2))+" % of AF patients got PEEP")
print(str(np.round(len(numeric_pd_patients[(numeric_pd_patients.AF==0)&(numeric_pd_patients.value>peep_setting)].admissionid.unique())/len(AF_admission_dataset[AF_admission_dataset.AF==0].admissionid.unique())*100,2))+" % of no AF patients got PEEP")
print("So AF patients got "+str(np.round(np.round(len(numeric_pd_patients[numeric_pd_patients.AF==1].admissionid.unique())
                                      /len(AF_admission_dataset[AF_admission_dataset.AF==1].admissionid.unique())*100,2)
                             /np.round(len(numeric_pd_patients[(numeric_pd_patients.AF==0)&(numeric_pd_patients.value>peep_setting)].admissionid.unique())
                                           /len(AF_admission_dataset[AF_admission_dataset.AF==0].admissionid.unique())*100,2),2))+" times more often at least once PEEP during admission")
print("")
print(str(np.round(len(numeric_pd_patients[(numeric_pd_patients.AF==1)&(numeric_pd_patients.value>peep_setting)].admissionid.unique())/len(numeric_pd_patients[(numeric_pd_patients.value>peep_setting)].admissionid.unique())*100,2))+" % of PEEP patients had AF")
print(str(np.round(len(numeric_pd_patients[(numeric_pd_patients.AF==0)&(numeric_pd_patients.value>peep_setting)].admissionid.unique())/len(numeric_pd_patients[(numeric_pd_patients.value>peep_setting)].admissionid.unique())*100,2))+" % of PEEP patients had no AF")
print(str(np.round(len(AF_admission_dataset[AF_admission_dataset.AF==1].admissionid.unique())/len(AF_admission_dataset.admissionid.unique())*100,2))+" % of ALL patients had AF")
print(str(np.round(len(numeric_pd_patients[(numeric_pd_patients.value>peep_setting)].admissionid.unique())/len(AF_admission_dataset.admissionid.unique())*100,2))+" % of ALL patients received PEEP")

In [ ]:
numeric_pd_patients_AF = numeric_pd_patients[numeric_pd_patients.AF==1][["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max'}).reset_index()
numeric_pd_patients_NO_AF = numeric_pd_patients[numeric_pd_patients.AF==0][["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max'}).reset_index()

print("AF average PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_AF.value["mean"])))
print("NO AF average PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_NO_AF.value["mean"])))
print("P-value difference between both distributions: "+str(ranksums(numeric_pd_patients_NO_AF.value["mean"],numeric_pd_patients_AF.value["mean"])[1]))
print(40*"-")
print("AF MIN PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_AF.value["min"])))
print("NO AF MIN PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_NO_AF.value["min"])))
print("P-value difference between both distributions: "+str(ranksums(numeric_pd_patients_NO_AF.value["min"],numeric_pd_patients_AF.value["min"])[1]))
print(40*"-")
print("AF MAX PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_AF.value["max"])))
print("NO AF MAX PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_NO_AF.value["max"])))
print("P-value difference between both distributions: "+str(ranksums(numeric_pd_patients_NO_AF.value["max"],numeric_pd_patients_AF.value["max"])[1]))
print(40*"-")
plt.hist(numeric_pd_patients_AF.value["mean"],label="AF",density=True,bins=np.arange(0.5,25,1))
plt.hist(numeric_pd_patients_NO_AF.value["mean"],label="NO AF",density=True,bins=np.arange(0.5,25,1),alpha=0.5)
plt.legend()

In [ ]:
numeric_pd_patients_AF = numeric_pd_patients[numeric_pd_patients.AF==1][["admissionid","itemid","value","measuredat_min"]].groupby(["admissionid","itemid"]).apply(lambda x:linregress(x.measuredat_min,x.value)[0]).reset_index()
numeric_pd_patients_AF.columns = [str(col) for col in numeric_pd_patients_AF.columns.values]
numeric_pd_patients_AF = numeric_pd_patients_AF.rename(columns={'0':"slope"})
numeric_pd_patients_AF = numeric_pd_patients_AF[~(numeric_pd_patients_AF.slope.isna())]

numeric_pd_patients_NO_AF = numeric_pd_patients[numeric_pd_patients.AF==0][["admissionid","itemid","value","measuredat_min"]].groupby(["admissionid","itemid"]).apply(lambda x:linregress(x.measuredat_min,x.value)[0]).reset_index()
numeric_pd_patients_NO_AF.columns = [str(col) for col in numeric_pd_patients_NO_AF.columns.values]
numeric_pd_patients_NO_AF = numeric_pd_patients_NO_AF.rename(columns={'0':"slope"})
numeric_pd_patients_NO_AF = numeric_pd_patients_NO_AF[~(numeric_pd_patients_NO_AF.slope.isna())]

print("AF Slope of PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_AF.slope)))
print("NO AF Slope of  PEEP setting across patients "+str(hours_to_first_AF)+" hours before measurement: "+str(np.mean(numeric_pd_patients_NO_AF.slope)))
print("P-value difference between both distributions: "+str(ttest_ind(numeric_pd_patients_NO_AF.slope,numeric_pd_patients_AF.slope)[1]))

plt.hist(numeric_pd_patients_AF.slope,label="AF",bins=np.arange(-0.105,0.105,0.01),density=True)
plt.hist(numeric_pd_patients_NO_AF.slope,label="NO AF",bins=np.arange(-0.105,0.105,0.01),density=True,alpha=0.5)
plt.legend()

In [ ]:
numeric_pd_patients_agg = numeric_pd_patients[["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max'}).reset_index()
numeric_pd_patients_agg.itemid = numeric_pd_patients_agg.itemid.astype(str)
numeric_pd_patients_agg.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_agg.columns.values]
numeric_pd_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in numeric_pd_patients_agg.columns.values]
numeric_pd_patients_agg = numeric_pd_patients_agg.pivot(index='admissionid', columns='itemid')
numeric_pd_patients_agg.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_agg.columns.values]

numeric_pd_patients_slope = numeric_pd_patients[["admissionid","itemid","measuredat_min","value"]].groupby(["admissionid","itemid"]).apply(lambda x:linregress(x.measuredat_min,x.value)[0]).reset_index()
numeric_pd_patients_slope.columns = [str(col) for col in numeric_pd_patients_slope.columns.values]
numeric_pd_patients_slope = numeric_pd_patients_slope.rename(columns={'0':"slope"})
numeric_pd_patients_slope.itemid = numeric_pd_patients_slope.itemid.astype(str)
numeric_pd_patients_slope = numeric_pd_patients_slope.pivot(index='admissionid', columns='itemid')
numeric_pd_patients_slope.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_slope.columns.values]
numeric_pd_patients_slope = numeric_pd_patients_slope.reset_index()


#### AF training set

In [ ]:
#specify hours before AF. IF YOU WANT A 12 HOUR EXTRA WINDOW, specify margin_time as 0, if you want the 1.5 hour model, specify margin_time as 1.5 and this as margin time.
#Min of time_shift is equal to margin_time
#To run this code, make sure you have enough RAM, restart the whole notebook and run this
one_half_hour_model = False
# non_biased_model = False
six_hour_model = True

only_data_of_first_hours = False #Use only data of the first X hours of admission
include_AF_patients_without_AF_at_sample = True #Include moments in time of AF patients before the AF diagnosis in the dataset.

if one_half_hour_model:
    time_shift = 1.5
    margin_time = 1.5
else:
    time_shift = 12
    margin_time = 0

if six_hour_model:
    time_shift = 6
    margin_time = 0

to_hour_multiplier = 1000*60*60

np.random.seed(42)

total_window = (hours_to_first_AF+time_shift)#*60 #in minutes
print(total_window)

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=total_window)].copy(deep=True)
admission_pd_min_hour["date_corresponds_to_AF_admid"] = admission_pd_min_hour.admissionid

admission_pd_min_hour = admission_pd_min_hour.sample(len(admission_pd_min_hour),random_state=42)
admission_pd_min_hour = admission_pd_min_hour[(admission_pd_min_hour.AF==0)|(admission_pd_min_hour.AF_measuredat>=to_hour_multiplier*total_window)]

# if non_biased_model:
AF_measuredat_sample_df = admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat>to_hour_multiplier*total_window)][["admissionid","AF_measuredat"]].copy(deep=True)
for admissionid in admission_pd_min_hour[admission_pd_min_hour.AF==0].admissionid.values:
    if len(AF_measuredat_sample_df) == 0:
        break
    else:
        if len(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((admission_pd_min_hour[admission_pd_min_hour.admissionid==admissionid]['lengthofstay'].values[0])*to_hour_multiplier)])>0:
            random_state_admission = np.random.RandomState(admissionid)
            choice = random_state_admission.choice(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((admission_pd_min_hour[admission_pd_min_hour.admissionid==admissionid]['lengthofstay'].values[0]+0.1)*to_hour_multiplier)]['admissionid'].values)
            admission_pd_min_hour.loc[admission_pd_min_hour.admissionid==admissionid,"AF_measuredat"] = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid==choice].AF_measuredat.values[0]
            admission_pd_min_hour.loc[admission_pd_min_hour.admissionid==admissionid,"date_corresponds_to_AF_admid"] = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid==choice].admissionid.values[0]
            AF_measuredat_sample_df = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid!=choice]   

admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: to_hour_multiplier*np.random.randint(total_window,row['lengthofstay']+1) if (row['AF']==0) and ((pd.isnull(row["AF_measuredat"])|(row["AF_measuredat"]==0))) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     

# else:
    # admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: to_hour_multiplier*np.random.randint(total_window,row['lengthofstay']+1) if (row['AF']==0) and ((pd.isnull(row["AF_measuredat"])|(row["AF_measuredat"]==0))) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     

# print(len(admission_pd_min_hour[admission_pd_min_hour.AF==1]))
# print(len(admission_pd_min_hour[(admission_pd_min_hour.AF==0)&((admission_pd_min_hour.date_corresponds_to_AF_admid!=admission_pd_min_hour.admissionid))][["AF","admissionid","AF_measuredat","date_corresponds_to_AF_admid"]]))

admission_pd_min_hour["AF_orig"]=admission_pd_min_hour.AF
#AF_admission_dataset = first_AFs_pd_timed.append(no_AF_same_length).reset_index()
AF_admission_dataset = admission_pd_min_hour[admission_pd_min_hour.AF_measuredat>=to_hour_multiplier*total_window]#pd.concat([first_AFs_pd_timed,temp_pd]).reset_index(drop=True) #AF_dataset

# if only_data_of_first_hours:
#     AF_admission_dataset["AF_measuredat"] = to_hour_multiplier*total_window #only take data of the first 48 hours

In [ ]:
def linreg_except(x,value,measuredat):
    try:
        return linregress(x[measuredat],x[value])[0]
    except:
        return np.nan

In [ ]:
#NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
numeric_pd_patients = (numeric_items_pd[numeric_items_pd.admissionid.isin(AF_admission_dataset.admissionid)]).merge(AF_admission_dataset[["admissionid","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='admissionid')
numeric_pd_patients["time_to_AF"]=(numeric_pd_patients.AF_measuredat.values-to_hour_multiplier*margin_time) - numeric_pd_patients.measuredat.values #add one margin_time to AF extra
numeric_pd_patients = numeric_pd_patients[(numeric_pd_patients.time_to_AF > (time_shift-margin_time)*to_hour_multiplier) & (numeric_pd_patients.time_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1) )]

for itemid_loop in numeric_items_pd.itemid.unique():
    numeric_pd_patients.itemid = numeric_pd_patients.itemid.replace(itemid_loop,dictionary[dictionary.itemid==itemid_loop].item.values[0])

numeric_items_pd = None #RAM Optimization

print("aggregating numeric all")

#numeric_pd_patients_agg = numeric_pd_patients[["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max','std','skew',pd.DataFrame.kurt}).reset_index()
numeric_pd_patients_agg = numeric_pd_patients[["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max',pd.DataFrame.kurt}).reset_index()
numeric_pd_patients_agg.itemid = numeric_pd_patients_agg.itemid.astype(str)
numeric_pd_patients_agg.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_agg.columns.values]
numeric_pd_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in numeric_pd_patients_agg.columns.values]
numeric_pd_patients_agg = numeric_pd_patients_agg.pivot(index='admissionid', columns='itemid')
numeric_pd_patients_agg.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_agg.columns.values]

print("aggregating numeric slope")

numeric_pd_patients_slope = numeric_pd_patients[["admissionid","itemid","measuredat_min","value"]].groupby(["admissionid","itemid"]).apply(lambda x:linreg_except(x,"value","measuredat_min")).reset_index()
numeric_pd_patients_slope.columns = [str(col) for col in numeric_pd_patients_slope.columns.values]
numeric_pd_patients_slope = numeric_pd_patients_slope.rename(columns={'0':"slope"})
numeric_pd_patients_slope.itemid = numeric_pd_patients_slope.itemid.astype(str)
numeric_pd_patients_slope = numeric_pd_patients_slope.pivot(index='admissionid', columns='itemid')
numeric_pd_patients_slope.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_slope.columns.values]
numeric_pd_patients_slope = numeric_pd_patients_slope.reset_index()

numeric_pd_patients_total = numeric_pd_patients_agg.merge(numeric_pd_patients_slope,how='left',on='admissionid')
numeric_pd_patients = None #Save RAM
numeric_pd_patients_slope = None
numeric_pd_patients_agg = None

print("Ready with numeric")
print("aggregating drug all")

#DRUG PREPROCESSING
drug_items_pd["administered_rate_min"] = drug_items_pd["administered"]/drug_items_pd["duration"]
drug_items_pd["measuredat_min_avg"] = (drug_items_pd["start"]+drug_items_pd["stop"])/(2*(1000*60))

drug_items_patients = (drug_items_pd[drug_items_pd.admissionid.isin(AF_admission_dataset.admissionid)]).merge(AF_admission_dataset[["admissionid","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='admissionid')
drug_items_patients["time_stop_to_AF"]=(drug_items_patients.AF_measuredat.values-1000*60*60*margin_time) - drug_items_patients.stop.values #add one hour to AF extra
drug_items_patients["time_start_to_AF"]=(drug_items_patients.AF_measuredat.values-1000*60*60*margin_time) - drug_items_patients.start.values #add one hour to AF extra

drug_items_patients = drug_items_patients[((drug_items_patients.time_stop_to_AF > (time_shift-margin_time)*1000*60*60)) & (drug_items_patients.time_stop_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                          ((drug_items_patients.time_start_to_AF > (time_shift-margin_time)*1000*60*60)) & (drug_items_patients.time_start_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                          ((drug_items_patients.time_start_to_AF <= (time_shift-margin_time)*1000*60*60)) & (drug_items_patients.time_stop_to_AF/1000/60/60 >= (time_shift+hours_to_first_AF-margin_time-1 ))]

for itemid_loop in drug_items_pd.itemid.unique():
    drug_items_patients.itemid = drug_items_patients.itemid.replace(itemid_loop,dictionary[dictionary.itemid==itemid_loop].item.values[0])

drug_items_agg = drug_items_patients[["admissionid","itemid","administered_rate_min"]].groupby(["admissionid","itemid"]).agg({'mean','min','max'}).reset_index()
drug_items_agg.itemid = drug_items_agg.itemid.astype(str)
drug_items_agg.columns = ['_'.join(col).rstrip('_') for col in drug_items_agg.columns.values]
drug_items_agg.columns = [col.replace('administered_rate_min_','') if 'administered_rate_min_' in col else col for col in drug_items_agg.columns.values]
drug_items_agg = drug_items_agg.pivot(index='admissionid', columns='itemid')
drug_items_agg.columns = ['_'.join(col).rstrip('_') for col in drug_items_agg.columns.values]

print("aggregating drug slope")

drug_items_patients_slope = drug_items_patients[["admissionid","itemid","measuredat_min_avg","administered_rate_min"]].groupby(["admissionid","itemid"]).apply(lambda x:linreg_except(x,"administered_rate_min","measuredat_min_avg")).reset_index()
drug_items_patients_slope.columns = [str(col) for col in drug_items_patients_slope.columns.values]
drug_items_patients_slope = drug_items_patients_slope.rename(columns={'0':"slope"})
drug_items_patients_slope.itemid = drug_items_patients_slope.itemid.astype(str)
drug_items_patients_slope = drug_items_patients_slope.pivot(index='admissionid', columns='itemid')
drug_items_patients_slope.columns = ['_'.join(col).rstrip('_') for col in drug_items_patients_slope.columns.values]
drug_items_patients_slope = drug_items_patients_slope.reset_index()

drug_items_total = drug_items_agg.merge(drug_items_patients_slope,how='left',on='admissionid')

print("Ready with drug items")

numeric_fb_patients = (numeric_fluidbalance_pd[numeric_fluidbalance_pd.admissionid.isin(AF_admission_dataset.admissionid)]).merge(AF_admission_dataset[["admissionid","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='admissionid')
numeric_fb_patients["time_to_AF"]=(numeric_fb_patients.AF_measuredat.values-1000*60*60*margin_time) - numeric_fb_patients.measuredat.values #add one hour to AF extra
numeric_fb_patients = numeric_fb_patients[(numeric_fb_patients.time_to_AF > (time_shift-margin_time)*1000*60*60) & (numeric_fb_patients.time_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1) )]

numeric_fb_patients = numeric_fb_patients[["admissionid","fluidout"]].groupby(["admissionid"]).sum().reset_index()

drug_fb_patients = (drugitems_fluidbalance_pd[drugitems_fluidbalance_pd.admissionid.isin(AF_admission_dataset.admissionid)]).merge(AF_admission_dataset[["admissionid","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='admissionid')
drug_fb_patients["time_stop_to_AF"]=(drug_fb_patients.AF_measuredat.values-1000*60*60*margin_time) - drug_fb_patients.stop.values #add one hour to AF extra
drug_fb_patients["time_start_to_AF"]=(drug_fb_patients.AF_measuredat.values-1000*60*60*margin_time) - drug_fb_patients.start.values #add one hour to AF extra

drug_fb_patients = drug_fb_patients[((drug_fb_patients.time_stop_to_AF > (time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_stop_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                    ((drug_fb_patients.time_start_to_AF > (time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_start_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                    ((drug_fb_patients.time_start_to_AF <= (time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_stop_to_AF/1000/60/60 >= (time_shift+hours_to_first_AF-margin_time-1 ))]

drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF<=((time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_stop_to_AF/1000/60/60>= (time_shift+hours_to_first_AF-margin_time-1)),"fluidin"]=((time_shift+hours_to_first_AF-margin_time-1)*1000*60*60)/(drug_fb_patients.duration*1000*60)*drug_fb_patients.fluidin 
drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF<((time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_stop_to_AF/1000/60/60<= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF > ((time_shift-margin_time)*1000*60*60)) ,"fluidin"]=drug_fb_patients.time_stop_to_AF/(drug_fb_patients.duration*1000*60)*drug_fb_patients.fluidin 
drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF>=((time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_start_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF/1000/60/60<= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF > ((time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.rate==0) & (drug_fb_patients.dose!=0) & (drug_fb_patients.solutionadministered!=0) & (drug_fb_patients.duration==1) ,"fluidin"]=drug_fb_patients.solutionadministered
drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF>=((time_shift-margin_time)*1000*60*60)) & (drug_fb_patients.time_start_to_AF/1000/60/60 <= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF/1000/60/60> (time_shift+hours_to_first_AF-margin_time-1)),"fluidin"]=((time_shift+hours_to_first_AF-margin_time-1)*1000*60*60-drug_fb_patients.time_start_to_AF)/(drug_fb_patients.duration*1000*60)*drug_fb_patients.duration

drug_fb_agg = drug_fb_patients[["admissionid","fluidin"]].groupby(["admissionid"]).sum().reset_index()

print("Ready with fluid balance")

fb_agg = drug_fb_agg.merge(numeric_fb_patients,on='admissionid')
fb_agg["fluid_balance"]=fb_agg["fluidin"]-fb_agg["fluidout"]

AF_dataset = numeric_pd_patients_total.merge(AF_admission_dataset,how='left',on='admissionid')
AF_dataset = AF_dataset.merge(drug_items_total,how='left',on='admissionid')
AF_dataset = AF_dataset.merge(fb_agg,how='left',on='admissionid')
#AF_dataset[AF_dataset.columns.drop(["dateofdeath_delta","admittedat_delta"])] = AF_dataset[AF_dataset.columns.drop(["dateofdeath_delta","admittedat_delta"])].fillna(AF_dataset[AF_dataset.columns.drop(["dateofdeath_delta","admittedat_delta"])].mean())

AF_dataset["is_given_Noradrenaline (Norepinefrine)"]=0
AF_dataset.loc[AF_dataset['mean_Noradrenaline (Norepinefrine)']>0,"is_given_Noradrenaline (Norepinefrine)"]=1
AF_dataset["is_given_Fentanyl"]=0
AF_dataset.loc[AF_dataset['mean_Fentanyl']>0,"is_given_Fentanyl"]=1
AF_dataset["is_given_Furosemide (Lasix)"]=0
AF_dataset.loc[AF_dataset['mean_Furosemide (Lasix)']>0,"is_given_Furosemide (Lasix)"]=1

In [ ]:
if not six_hour_model:
    if one_half_hour_model:
        AF_dataset.to_csv(data_amsterdam_get_path+"Extracted/AF_dataset_1_5_hours_no_af_distribution_matched_bugfixed.csv",index=False)
    else:
        AF_dataset.to_csv(data_amsterdam_get_path+"Extracted/AF_dataset_12_hours_no_af_distribution_matched_bugfixed.csv",index=False)
else:
    AF_dataset.to_csv(data_amsterdam_get_path+"Extracted/AF_dataset_"+str(time_shift)+"_hours_no_af_distribution_matched_bugfixed.csv",index=False)

#### AF Risk over Time

##### Different distribution visualization of NO AF vs AF patients

In [ ]:
hours_to_first_AF = 12
min_time_difference_to_AF = 0

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(24,8))

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=hours_to_first_AF+min_time_difference_to_AF)].copy(deep=True)
admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: 1000*60*60*np.random.randint(hours_to_first_AF,row['lengthofstay']+1) if row['AF']==0 else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.
axs[0].hist(np.clip(admission_pd_min_hour[admission_pd_min_hour.AF==0].AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True)
axs[0].set_xlabel("Time after admission (hours) (clipped)")
axs[0].set_title("Random measurement point between 12 aft adm and discharge")

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=hours_to_first_AF+min_time_difference_to_AF)].copy(deep=True)
admission_pd_min_hour_AF_filtered = admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat>1000*60*60*hours_to_first_AF)]
admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: np.random.choice(admission_pd_min_hour_AF_filtered[admission_pd_min_hour_AF_filtered.AF_measuredat<=((row['lengthofstay']+0.1)*1000*60*60)]['AF_measuredat'].values) if row['AF']==0 else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.
axs[1].hist(np.clip(admission_pd_min_hour[admission_pd_min_hour.AF==0].AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True)
axs[1].set_xlabel("Time after admission (hours) (clipped)")
axs[1].set_title("Random measurement point chosen from available AF measurements")

axs[2].hist(np.clip(admission_pd_min_hour_AF_filtered.AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True)
axs[2].set_xlabel("Time after admission (hours) (clipped)")
axs[2].set_title("AF distribution")

if present:
    axs[0].patch.set_alpha(0)
    axs[1].patch.set_alpha(0)
    axs[2].patch.set_alpha(0)

In [ ]:
hours_to_first_AF = 12
min_time_difference_to_AF = 0

fig,axs = plt.subplots(1,2,figsize=(24,8))

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=hours_to_first_AF+min_time_difference_to_AF)].copy(deep=True)
admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: 1000*60*60*np.random.randint(hours_to_first_AF,row['lengthofstay']+1) if row['AF']==0 else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.
axs[0].hist(np.clip(admission_pd_min_hour[admission_pd_min_hour.AF==0].AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True,color="#e898ac",ec="white")
axs[0].set_xlabel("Time after admission (hours) (clipped)")
axs[0].set_title("NO AF distribution")
axs[0].set_xticks([0,10,20,30,40,50,60,70,80,90,100])

axs[1].hist(np.clip(admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat/1000/60/60>=hours_to_first_AF+min_time_difference_to_AF)].AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True,color="#e898ac",ec="white")
axs[1].set_xlabel("Time after admission (hours) (clipped)")
axs[1].set_title("AF distribution")
axs[1].set_xticks([0,10,20,30,40,50,60,70,80,90,100])

if present:
    axs[0].patch.set_alpha(0)
    axs[1].patch.set_alpha(0)

In [ ]:
admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=hours_to_first_AF+min_time_difference_to_AF)].copy(deep=True)
AF_measuredat_sample_df = admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat>1000*60*60*hours_to_first_AF)][["admissionid","AF_measuredat"]].copy(deep=True)
admission_pd_min_hour["date_corresponds_to_AF_admid"] = admission_pd_min_hour.admissionid
for admissionid in admission_pd_min_hour[admission_pd_min_hour.AF==0].admissionid.values:
    if len(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((admission_pd_min_hour[admission_pd_min_hour.admissionid==admissionid]['lengthofstay'].values[0])*1000*60*60)])>0:
        choice = np.random.choice(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((admission_pd_min_hour[admission_pd_min_hour.admissionid==admissionid]['lengthofstay'].values[0]+0.1)*1000*60*60)]['admissionid'].values)
        admission_pd_min_hour.loc[admission_pd_min_hour.admissionid==admissionid,"AF_measuredat"] = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid==choice].AF_measuredat.values[0]
        admission_pd_min_hour.loc[admission_pd_min_hour.admissionid==admissionid,"date_corresponds_to_AF_admid"] = choice
        AF_measuredat_sample_df = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid!=choice]   
admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: 1000*60*60*np.random.randint(hours_to_first_AF,row['lengthofstay']+1) if (row['AF']==0) and (row['date_corresponds_to_AF_admid']==row['admissionid']) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     
            
plt.figure()
plt.hist(np.clip(admission_pd_min_hour[(admission_pd_min_hour.AF==0)&(admission_pd_min_hour.date_corresponds_to_AF_admid!=admission_pd_min_hour.admissionid)].AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True)
plt.xlabel("Time after admission (hours) (clipped)")
plt.title("Random measurement point chosen from available AF measurements") 
plt.plot()

plt.figure()
plt.hist(np.clip(admission_pd_min_hour[(admission_pd_min_hour.AF==0)&(admission_pd_min_hour.date_corresponds_to_AF_admid==admission_pd_min_hour.admissionid)].AF_measuredat/1000/60/60,0,100),bins=np.arange(0,101,5),density=True)
plt.xlabel("Time after admission (hours) (clipped)")
plt.title("Random measurement point chosen from available AF measurements") 

##### Dataset generation

In [19]:
def linreg_except(x,value,measuredat):
    try:
        return linregress(x[measuredat],x[value])[0]
    except:
        return np.nan

In [20]:
from scipy.stats import linregress

use_X_hours_after_admission = True #add absolute hours to admission time until the time exceeds or is equal to the prediction point
use_X_percentage_after_admission = False #make windows based on the percentage time already on ICU, with 100% the time to the prediction point, if false, substract absolute hours from measuredat

one_half_hour_model = True

if one_half_hour_model:
    time_shift = 1.5
    margin_time = 1.5
else:
    time_shift = 12
    margin_time = 0

to_hour_multiplier = 1000*60*60

#AF patients but the timestamp is not of the AF diagnosis
#min_time_difference_to_AF = 4 #The amount of hours before AF

np.random.seed(42)
total_window = (hours_to_first_AF+time_shift)#*60 #in minutes

admission_pd_min_hour = admissions_pd[(admissions_pd.lengthofstay>=total_window)].copy(deep=True)
admission_pd_min_hour["date_corresponds_to_AF_admid"] = admission_pd_min_hour.admissionid

admission_pd_min_hour = admission_pd_min_hour.sample(len(admission_pd_min_hour),random_state=42)
admission_pd_min_hour = admission_pd_min_hour[(admission_pd_min_hour.AF==0)|(admission_pd_min_hour.AF_measuredat>=to_hour_multiplier*total_window)]


AF_measuredat_sample_df = admission_pd_min_hour[(admission_pd_min_hour.AF==1)&(admission_pd_min_hour.AF_measuredat>to_hour_multiplier*total_window)][["admissionid","AF_measuredat"]].copy(deep=True)
for admissionid in admission_pd_min_hour[admission_pd_min_hour.AF==0].admissionid.values:
    if len(AF_measuredat_sample_df) == 0:
        break
    else:
        if len(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((admission_pd_min_hour[admission_pd_min_hour.admissionid==admissionid]['lengthofstay'].values[0])*to_hour_multiplier)])>0:
            random_state_admission = np.random.RandomState(admissionid)
            choice = random_state_admission.choice(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((admission_pd_min_hour[admission_pd_min_hour.admissionid==admissionid]['lengthofstay'].values[0]+0.1)*to_hour_multiplier)]['admissionid'].values)
            admission_pd_min_hour.loc[admission_pd_min_hour.admissionid==admissionid,"AF_measuredat"] = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid==choice].AF_measuredat.values[0]
            admission_pd_min_hour.loc[admission_pd_min_hour.admissionid==admissionid,"date_corresponds_to_AF_admid"] = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid==choice].admissionid.values[0]
            AF_measuredat_sample_df = AF_measuredat_sample_df[AF_measuredat_sample_df.admissionid!=choice] 

admission_pd_min_hour.loc[:,"AF_measuredat"] =  admission_pd_min_hour.apply(lambda row: to_hour_multiplier*np.random.randint(total_window,row['lengthofstay']+1) if (row['AF']==0) and ((pd.isnull(row["AF_measuredat"])|(row["AF_measuredat"]==0))) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     
admission_pd_min_hour["AF_orig"]=admission_pd_min_hour.AF

#Because min time difference changes every loop the seed for random changes and we don't want that
admission_pd_min_hour = admission_pd_min_hour[(admission_pd_min_hour.lengthofstay>=total_window)].copy(deep=True)

#Check whether the time window does not exceed the prediction point
no_AF_but_AFs_pd_timed = admission_pd_min_hour[(admission_pd_min_hour.AF_measuredat/to_hour_multiplier>=hours_to_first_AF+margin_time)].reset_index(drop=True)

for min_time_difference_to_AF in np.arange(0,101,4):#np.arange(0,101,4):
    print(min_time_difference_to_AF)

    if use_X_hours_after_admission:
        #add absolute hours to admission time until the time exceeds or is equal to the prediction point
        no_AF_but_AFs_pd_timed["rot_AF_measuredat"] = to_hour_multiplier*(hours_to_first_AF+margin_time+min_time_difference_to_AF)
        no_AF_but_AFs_pd_timed = no_AF_but_AFs_pd_timed[no_AF_but_AFs_pd_timed.rot_AF_measuredat <= no_AF_but_AFs_pd_timed.AF_measuredat]

    else: 
        if use_X_percentage_after_admission:
            #make windows based on the percentage time already on ICU, with 100% the time to the prediction point
            no_AF_but_AFs_pd_timed["rot_AF_measuredat"] = no_AF_but_AFs_pd_timed.AF_measuredat*(min_time_difference_to_AF/100)
        else:
            #substract absolute hours from measuredat
            no_AF_but_AFs_pd_timed["rot_AF_measuredat"] = no_AF_but_AFs_pd_timed.AF_measuredat-to_hour_multiplier*min_time_difference_to_AF

    #if the prediction point is within 30 minutes of the actual prediction point, we use the same label
    no_AF_but_AFs_pd_timed.loc[no_AF_but_AFs_pd_timed.AF_measuredat - no_AF_but_AFs_pd_timed.rot_AF_measuredat >= to_hour_multiplier/2,"AF"] = 0

    #NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
    numeric_pd_patients = (numeric_items_pd[numeric_items_pd.admissionid.isin(no_AF_but_AFs_pd_timed.admissionid)]).merge(no_AF_but_AFs_pd_timed[["admissionid","rot_AF_measuredat"]],how='left',on='admissionid')
    numeric_pd_patients["time_to_AF"]=(numeric_pd_patients.rot_AF_measuredat.values-to_hour_multiplier*margin_time) - numeric_pd_patients.measuredat.values #add one margin_time to AF extra
    numeric_pd_patients = numeric_pd_patients[(numeric_pd_patients.time_to_AF > (time_shift-margin_time)*to_hour_multiplier) & (numeric_pd_patients.time_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1) )]

    for itemid_loop in numeric_items_pd.itemid.unique():
        numeric_pd_patients.itemid = numeric_pd_patients.itemid.replace(itemid_loop,dictionary[dictionary.itemid==itemid_loop].item.values[0])

    #numeric_items_pd = None #RAM Optimization

    #numeric_pd_patients_agg = numeric_pd_patients[["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max','std','skew',pd.DataFrame.kurt}).reset_index()
    numeric_pd_patients_agg = numeric_pd_patients[["admissionid","itemid","value"]].groupby(["admissionid","itemid"]).agg({'mean','min','max',pd.DataFrame.kurt}).reset_index()
    numeric_pd_patients_agg.itemid = numeric_pd_patients_agg.itemid.astype(str)
    numeric_pd_patients_agg.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_agg.columns.values]
    numeric_pd_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in numeric_pd_patients_agg.columns.values]
    numeric_pd_patients_agg = numeric_pd_patients_agg.pivot(index='admissionid', columns='itemid')
    numeric_pd_patients_agg.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_agg.columns.values]

    numeric_pd_patients_slope = numeric_pd_patients[["admissionid","itemid","measuredat_min","value"]].groupby(["admissionid","itemid"]).apply(lambda x:linreg_except(x,"value","measuredat_min")).reset_index()
    numeric_pd_patients_slope.columns = [str(col) for col in numeric_pd_patients_slope.columns.values]
    numeric_pd_patients_slope = numeric_pd_patients_slope.rename(columns={'0':"slope"})
    numeric_pd_patients_slope.itemid = numeric_pd_patients_slope.itemid.astype(str)
    numeric_pd_patients_slope = numeric_pd_patients_slope.pivot(index='admissionid', columns='itemid')
    numeric_pd_patients_slope.columns = ['_'.join(col).rstrip('_') for col in numeric_pd_patients_slope.columns.values]
    numeric_pd_patients_slope = numeric_pd_patients_slope.reset_index()

    numeric_pd_patients_total = numeric_pd_patients_agg.merge(numeric_pd_patients_slope,how='left',on='admissionid')
    numeric_pd_patients = None #Save RAM
    numeric_pd_patients_slope = None
    numeric_pd_patients_agg = None

    #DRUG PREPROCESSING
    drug_items_pd["administered_rate_min"] = drug_items_pd["administered"]/drug_items_pd["duration"]
    drug_items_pd["measuredat_min_avg"] = (drug_items_pd["start"]+drug_items_pd["stop"])/(2*(to_hour_multiplier/60))

    drug_items_patients = (drug_items_pd[drug_items_pd.admissionid.isin(no_AF_but_AFs_pd_timed.admissionid)]).merge(no_AF_but_AFs_pd_timed[["admissionid","rot_AF_measuredat"]],how='left',on='admissionid')
    drug_items_patients["time_stop_to_AF"]=(drug_items_patients.rot_AF_measuredat.values-to_hour_multiplier*margin_time) - drug_items_patients.stop.values #add one hour to AF extra
    drug_items_patients["time_start_to_AF"]=(drug_items_patients.rot_AF_measuredat.values-to_hour_multiplier*margin_time) - drug_items_patients.start.values #add one hour to AF extra

    drug_items_patients = drug_items_patients[((drug_items_patients.time_stop_to_AF > (time_shift-margin_time)*to_hour_multiplier)) & (drug_items_patients.time_stop_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                              ((drug_items_patients.time_start_to_AF > (time_shift-margin_time)*to_hour_multiplier)) & (drug_items_patients.time_start_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                              ((drug_items_patients.time_start_to_AF <= (time_shift-margin_time)*to_hour_multiplier)) & (drug_items_patients.time_stop_to_AF/to_hour_multiplier >= (time_shift+hours_to_first_AF-margin_time-1 ))]

    for itemid_loop in drug_items_pd.itemid.unique():
        drug_items_patients.itemid = drug_items_patients.itemid.replace(itemid_loop,dictionary[dictionary.itemid==itemid_loop].item.values[0])

    drug_items_agg = drug_items_patients[["admissionid","itemid","administered_rate_min"]].groupby(["admissionid","itemid"]).agg({'mean','min','max'}).reset_index()
    drug_items_agg.itemid = drug_items_agg.itemid.astype(str)
    drug_items_agg.columns = ['_'.join(col).rstrip('_') for col in drug_items_agg.columns.values]
    drug_items_agg.columns = [col.replace('administered_rate_min_','') if 'administered_rate_min_' in col else col for col in drug_items_agg.columns.values]
    drug_items_agg = drug_items_agg.pivot(index='admissionid', columns='itemid')
    drug_items_agg.columns = ['_'.join(col).rstrip('_') for col in drug_items_agg.columns.values]

    drug_items_patients_slope = drug_items_patients[["admissionid","itemid","measuredat_min_avg","administered_rate_min"]].groupby(["admissionid","itemid"]).apply(lambda x:linreg_except(x,"administered_rate_min","measuredat_min_avg")).reset_index()
    drug_items_patients_slope.columns = [str(col) for col in drug_items_patients_slope.columns.values]
    drug_items_patients_slope = drug_items_patients_slope.rename(columns={'0':"slope"})
    drug_items_patients_slope.itemid = drug_items_patients_slope.itemid.astype(str)
    drug_items_patients_slope = drug_items_patients_slope.pivot(index='admissionid', columns='itemid')
    drug_items_patients_slope.columns = ['_'.join(col).rstrip('_') for col in drug_items_patients_slope.columns.values]
    drug_items_patients_slope = drug_items_patients_slope.reset_index()

    drug_items_total = drug_items_agg.merge(drug_items_patients_slope,how='left',on='admissionid')

    numeric_fb_patients = (numeric_fluidbalance_pd[numeric_fluidbalance_pd.admissionid.isin(no_AF_but_AFs_pd_timed.admissionid)]).merge(no_AF_but_AFs_pd_timed[["admissionid","rot_AF_measuredat"]],how='left',on='admissionid')
    numeric_fb_patients["time_to_AF"]=(numeric_fb_patients.rot_AF_measuredat.values-to_hour_multiplier*margin_time) - numeric_fb_patients.measuredat.values #add one hour to AF extra
    numeric_fb_patients = numeric_fb_patients[(numeric_fb_patients.time_to_AF > (time_shift-margin_time)*to_hour_multiplier) & (numeric_fb_patients.time_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1) )]

    numeric_fb_patients = numeric_fb_patients[["admissionid","fluidout"]].groupby(["admissionid"]).sum().reset_index()

    drug_fb_patients = (drugitems_fluidbalance_pd[drugitems_fluidbalance_pd.admissionid.isin(no_AF_but_AFs_pd_timed.admissionid)]).merge(no_AF_but_AFs_pd_timed[["admissionid","rot_AF_measuredat"]],how='left',on='admissionid')
    drug_fb_patients["time_stop_to_AF"]=(drug_fb_patients.rot_AF_measuredat.values-to_hour_multiplier*margin_time) - drug_fb_patients.stop.values #add one hour to AF extra
    drug_fb_patients["time_start_to_AF"]=(drug_fb_patients.rot_AF_measuredat.values-to_hour_multiplier*margin_time) - drug_fb_patients.start.values #add one hour to AF extra

    drug_fb_patients = drug_fb_patients[((drug_fb_patients.time_stop_to_AF > (time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_stop_to_AF/to_hour_multiplier<= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                        ((drug_fb_patients.time_start_to_AF > (time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_start_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1 ))|
                                        ((drug_fb_patients.time_start_to_AF <= (time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_stop_to_AF/to_hour_multiplier >= (time_shift+hours_to_first_AF-margin_time-1 ))]

    drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF<=((time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_stop_to_AF/to_hour_multiplier>= (time_shift+hours_to_first_AF-margin_time-1)),"fluidin"]=((time_shift+hours_to_first_AF-margin_time-1)*to_hour_multiplier)/(drug_fb_patients.duration*1000*60)*drug_fb_patients.fluidin 
    drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF<((time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_stop_to_AF/to_hour_multiplier<= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF > ((time_shift-margin_time)*to_hour_multiplier)) ,"fluidin"]=drug_fb_patients.time_stop_to_AF/(drug_fb_patients.duration*1000*60)*drug_fb_patients.fluidin 
    drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF>=((time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_start_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF/to_hour_multiplier<= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF > ((time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.rate==0) & (drug_fb_patients.dose!=0) & (drug_fb_patients.solutionadministered!=0) & (drug_fb_patients.duration==1) ,"fluidin"]=drug_fb_patients.solutionadministered
    drug_fb_patients.loc[(drug_fb_patients.time_start_to_AF>=((time_shift-margin_time)*to_hour_multiplier)) & (drug_fb_patients.time_start_to_AF/to_hour_multiplier <= (time_shift+hours_to_first_AF-margin_time-1)) & (drug_fb_patients.time_stop_to_AF/to_hour_multiplier> (time_shift+hours_to_first_AF-margin_time-1)),"fluidin"]=((time_shift+hours_to_first_AF-margin_time-1)*to_hour_multiplier-drug_fb_patients.time_start_to_AF)/(drug_fb_patients.duration*1000*60)*drug_fb_patients.duration

    drug_fb_agg = drug_fb_patients[["admissionid","fluidin"]].groupby(["admissionid"]).sum().reset_index()

    fb_agg = drug_fb_agg.merge(numeric_fb_patients,on='admissionid')
    fb_agg["fluid_balance"]=fb_agg["fluidin"]-fb_agg["fluidout"]

    AF_dataset = numeric_pd_patients_total.merge(no_AF_but_AFs_pd_timed,how='left',on='admissionid')
    AF_dataset = AF_dataset.merge(drug_items_total,how='left',on='admissionid')
    AF_dataset = AF_dataset.merge(fb_agg,how='left',on='admissionid')

    AF_dataset["is_given_Noradrenaline (Norepinefrine)"]=0
    AF_dataset.loc[AF_dataset['mean_Noradrenaline (Norepinefrine)']>0,"is_given_Noradrenaline (Norepinefrine)"]=1
    AF_dataset["is_given_Fentanyl"]=0
    AF_dataset.loc[AF_dataset['mean_Fentanyl']>0,"is_given_Fentanyl"]=1
    AF_dataset["is_given_Furosemide (Lasix)"]=0
    AF_dataset.loc[AF_dataset['mean_Furosemide (Lasix)']>0,"is_given_Furosemide (Lasix)"]=1

    if not use_X_hours_after_admission:
        if use_X_percentage_after_admission:
            if one_half_hour_model:
                AF_dataset.to_csv(data_amsterdam_save_path+"rot/AF_dataset_1_5_hours_risk_over_time_"+str(min_time_difference_to_AF)+"_percent_time_after_admission.csv",index=False)
            else:
                AF_dataset.to_csv(data_amsterdam_save_path+"rot/AF_dataset_12_hours_risk_over_time_"+str(min_time_difference_to_AF)+"_percent_time_after_admission.csv",index=False)
        else:
            if one_half_hour_model:
                AF_dataset.to_csv(data_amsterdam_save_path+"rot/AF_dataset_1_5_hours_risk_over_time_"+str(min_time_difference_to_AF)+"_hours_before_prediction_point.csv",index=False)
            else:
                AF_dataset.to_csv(data_amsterdam_save_path+"rot/AF_dataset_12_hours_risk_over_time_"+str(min_time_difference_to_AF)+"_hours_before_prediction_point.csv",index=False)
    else:
        if one_half_hour_model:
            AF_dataset.to_csv(data_amsterdam_save_path+"rot/AF_dataset_1_5_hours_risk_over_time_"+str(min_time_difference_to_AF)+"hours_after_adm.csv",index=False)
        else:
            AF_dataset.to_csv(data_amsterdam_save_path+"rot/AF_dataset_12_hours_risk_over_time_"+str(min_time_difference_to_AF)+"hours_after_adm.csv",index=False)
    

0


invalid value encountered in double_scalars
invalid value encountered in sqrt
invalid value encountered in double_scalars


4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
